# General Setup

The session crushed, so I have to redo certian parts. For this reason order of the code may not make a sense.

In [ ]:
source("/home/ridvan/scRIPT/settings.R")
source("/home/ridvan/scRIPT/utils.R")

In [ ]:
BPPARAM <- BiocParallel::bpparam()
BPPARAM$workers = 20

# Multi core using future - built in to seurat
plan("multicore", workers = 20)
options(future.globals.maxSize = 200 * 1024 ^ 3) # for 200 Gb RAM

getwd()

In [ ]:
obj <- readRDS('../../cmo_version3.rds')
obj

In [ ]:
io$output_rds_dir
atlas <- readRDS(paste0(io$output_rds_dir, 'extended_atlas_cellranger_features_seurat.rds'))
atlas

## Minor Adjustments for the ordering atlas features

In [ ]:
table(obj$v7k75pca_stage_predicted.id)

In [ ]:
# Specify the desired order of the levels
stage_levels <- c(
  "E6.5", "E7.0", "E7.25", "E7.5", "E7.75", "E8.0", "E8.25", "E8.5",
  "E8.75", "E9.0", "E9.25", "E9.5", "Mixed gastrulation"
)

# Create a new factor column in your meta.data with the specified order
obj@meta.data$v7k75pca_stage_predicted.id_num <-
  factor(obj$v7k75pca_stage_predicted.id, levels = stage_levels)

# Check that the levels are correct
table(obj@meta.data$v7k75pca_stage_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_stages <- obj@meta.data$v7k75pca_stage_predicted.id_num

In [ ]:
obj$lv_stages_score <- obj@meta.data$v7k75pca_stage_prediction.score.max

In [ ]:
obj$lv_stages_ch <- as.character(obj$lv_stages) 

In [ ]:
table(obj$v7k75pca_anatomy_predicted.id)

In [ ]:
# Specify the desired order (spelling corrected and capitalized)
anatomy_levels <- c(
  "Pooled", 
  "YS", 
  "EP", 
  "Anterior", 
  "Posterior", 
  "Anterior section", 
  "Medial section", 
  "Posterior section"
)

# Create new factor column with ordered levels
obj@meta.data$v7k75pca_anatomy_predicted.id_num <- factor(
  obj$v7k75pca_anatomy_predicted.id,
  levels = anatomy_levels
)

# Check the result
table(obj@meta.data$v7k75pca_anatomy_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_anatomy <- obj@meta.data$v7k75pca_anatomy_predicted.id_num

In [ ]:
obj$lv_anatomy_score <- obj@meta.data$v7k75pca_anatomy_prediction.score.max

In [ ]:
obj$lv_anatomy_ch <- as.character(obj$lv_anatomy) 

In [ ]:
table(obj$v7k75pca_somite_count_predicted.id)

In [ ]:
# Specify the desired level order (numbers as characters, pooled last)
somite_levels <- c(
  "6", "7", "8", "10", "12", "14", "15", "16", "18", 
  "20", "21", "22", "23", "24", "Pooled"
)

# Create new factor column with ordered levels
obj@meta.data$v7k75pca_somite_count_predicted.id_num <- factor(
  obj$v7k75pca_somite_count_predicted.id,
  levels = somite_levels
)

# Check the result
table(obj@meta.data$v7k75pca_somite_count_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_somite <- obj@meta.data$v7k75pca_somite_count_predicted.id_num

In [ ]:
obj$lv_somite_score <- obj@meta.data$v7k75pca_somite_count_prediction.score.max

In [ ]:
obj$lv_somite_ch <- as.character(obj$lv_somite) 

In [ ]:
obj$lv_cell_type_score <- obj@meta.data$v7k_pca75_prediction.score.max

In [ ]:
obj$lv_cell_type <- obj@meta.data$v7k_pca75_predicted.id

In [ ]:
obj$lv_cell_to_cell <- obj@meta.data$v7k75pca_cell_predicted.id
obj$lv_cell_to_cell_score <- obj@meta.data$v7k75pca_cell_prediction.score.max

# Simplifiying the naming on CMO

In [ ]:
table(obj$cell_type_clusters)

In [ ]:
table(obj$cell_type_groups)

In [ ]:
table(obj$cell_type_clusters)

In [ ]:
obj
lm_obj <- subset(obj, subset = cell_type_groups %in% "Late_Mesoderm" |  cell_type_clusters %in% 'Nascent Mesoderm')
lm_obj

In [ ]:
DimPlot(lm_obj, reduction = "UMAP", cols = r3dcol$cols_46, group.by = 'cell_type_subclusters')

In [ ]:
# Findvariable Features
lm_obj <- FindVariableFeatures(object = lm_obj, nfeatures = 4000)

In [ ]:
lm_obj <- ScaleData(object = lm_obj, 
                          features = VariableFeatures(lm_obj),
                          vars.to.regress = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.ribo", "G2M.Score", "S.Score"),
                          block.size = 20000)

In [ ]:
lm_obj <- RunPCA(lm_obj,npcs = 50, reduction.name = "lm_pca",
                       features =  VariableFeatures(lm_obj)
                      )

In [ ]:
# New harmony Umap
lm_obj <- RunUMAP(lm_obj, reduction = "lm_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "lm_umap1")
lm_obj <- RunUMAP(lm_obj, reduction = "lm_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "lm_umap2")
lm_obj <- RunUMAP(lm_obj, reduction = "lm_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "lm_umap3")

In [ ]:
DimPlot(lm_obj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters')
DimPlot(lm_obj, reduction = "lm_umap1", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters')
DimPlot(lm_obj, reduction = "lm_umap2", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters')
DimPlot(lm_obj, reduction = "lm_umap3", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters')

In [ ]:
obj$lv_subclusters

In [ ]:
DimPlot(lm_obj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'lv_subclusters',label = TRUE) + NoLegend()
DimPlot(lm_obj, reduction = "lm_umap3", cols = r3dcol$cols_94, group.by = 'lv_subclusters', label = TRUE) + NoLegend()

In [ ]:
DimPlot(lm_obj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters',label = TRUE) + NoLegend()
DimPlot(lm_obj, reduction = "lm_umap3", cols = r3dcol$cols_94, group.by = 'cell_type_subclusters', label = TRUE) + NoLegend()

In [ ]:
levels( obj$cell_type_subclusters)

In [ ]:
new_names <- c(
    "Naive Pluripotency","Epiblast","Primitive Streak 1","Amniotic Ectoderm 1","Amniotic Ectoderm 2","Amniotic Ectoderm 3","PGC","Primitive Streak 2","Anterior Primitive Streak",
    "Visceral Endoderm","Nascent Mesoderm","Lateral Plate Mesoderm",
    "Epicardium 1","Epicardium 2",
    "Mesenchyme 1","Mesenchyme 2",
    "Epicardium 3",
    "Limb Mesoderm 1","Limb Mesoderm 2",
    "Allantois 1","Allantois 4","Allantois 2","Allantois 3","Atf3-KO Enriched","HEP/BP","Erythroid 1","Erythroid 2","Erythroid 3","Erythroid 4","MkP 1","MEP","MkP 2",
    "Endothelium 1","Endothelium 2","Endocardium","Endothelium 3","EMP 1","EMP 2","EMP 3","EMP 4"
    ) 

In [ ]:
Idents(obj) <- obj$lv_subclusters

In [ ]:
levels(obj)

In [ ]:
names(new_names) <- levels(obj)

In [ ]:
new_names

In [ ]:
obj <- RenameIdents(obj, new_names)

In [ ]:
obj$new_subclusters <- Idents(obj)

In [ ]:
obj$new_subclusters_ch <- as.character(obj$new_subclusters)

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'UMAP', group.by = 'new_subclusters', label = T,label.size = 10, alpha=0.7, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
new_names <- c(
    "Naive Pluripotency","Epiblast","Primitive Streak","Amniotic Ectoderm","Amniotic Ectoderm","Amniotic Ectoderm","PGC","Primitive Streak","Anterior Primitive Streak",
    "Visceral Endoderm","Nascent Mesoderm","Lateral Plate Mesoderm",
    "Epicardium","Epicardium",
    "Mesenchyme","Mesenchyme",
    "Epicardium",
    "Limb Mesoderm","Limb Mesoderm",
    "Allantois","Allantois","Allantois","Allantois","Atf3-KO Enriched","HEP/BP","Erythroid","Erythroid","Erythroid","Erythroid","MkP","MEP","MkP",
    "Endothelium","Endothelium","Endocardium","Endothelium","EMP","EMP","EMP","EMP"
    ) 

In [ ]:
Idents(obj) <- obj$lv_subclusters

In [ ]:
levels(obj)

In [ ]:
names(new_names) <- levels(obj)

In [ ]:
new_names

In [ ]:
obj <- RenameIdents(obj, new_names)

In [ ]:
obj$new_subclusters_x <- Idents(obj)

In [ ]:
obj$new_subclusters_x_ch <- as.character(obj$new_subclusters)

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'UMAP', group.by = 'new_subclusters_x', label = T,label.size = 10, alpha=0.7, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
new_names <- c(
    "Naive Pluripotency","Epiblast","Primitive Streak","Amniotic Ectoderm","Amniotic Ectoderm","Amniotic Ectoderm","PGC","Primitive Streak","Visceral Endoderm",
    "Visceral Endoderm","Nascent Mesoderm","Mesenchyme and Epicardium",
    "Mesenchyme and Epicardium","Mesenchyme and Epicardium",
    "Mesenchyme and Epicardium","Mesenchyme and Epicardium",
    "Mesenchyme and Epicardium",
    "Limb Mesoderm and Allantois","Limb Mesoderm and Allantois",
    "Limb Mesoderm and Allantois","Limb Mesoderm and Allantois","Limb Mesoderm and Allantois","Limb Mesoderm and Allantois","Atf3-KO Enriched","HEP/BP","Erythroid","Erythroid","Erythroid","Erythroid","MkP","MkP","MkP",
    "Endothelium","Endothelium","Endothelium","Endothelium","EMP","EMP","EMP","EMP"
    ) 

In [ ]:
Idents(obj) <- obj$lv_subclusters

In [ ]:
levels(obj)

In [ ]:
names(new_names) <- levels(obj)

In [ ]:
new_names

In [ ]:
obj <- RenameIdents(obj, new_names)

In [ ]:
obj$new_clusters <- Idents(obj)

In [ ]:
obj$new_clustersch <- as.character(obj$new_subclusters)

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'UMAP', group.by = 'new_clusters', label = T,label.size = 10, alpha=0.7, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
dim(obj@reductions$PCA)

In [ ]:
# New harmony Umap
obj <- RunUMAP(obj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "umap1")
obj <- RunUMAP(obj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "umap2")
obj <- RunUMAP(obj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "umap3")

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'umap1', group.by = 'new_subclusters', label = T,label.size = 3, alpha=0.9, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'umap2', group.by = 'new_subclusters', label = T,label.size = 3, alpha=0.9, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
DimPlot(obj, reduction = 'umap3', group.by = 'new_subclusters', label = T,label.size = 3, alpha=0.9, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
saveRDS(obj, '../../cmo_version4.rds', compress = FALSE)

In [ ]:
saveRDS(lm_obj, '../../cmo_latemesoderm.rds')

In [ ]:
obj <- readRDS('../../cmo_version4.rds')

In [ ]:
obj

## Subsetting wt samples for the CMO object

In [ ]:
table(obj$Conditions_lv)

In [ ]:
obj

In [ ]:
wt_obj <- subset(obj, subset = Conditions_lv %in% "Control" )

In [ ]:
wt_obj

In [ ]:
table(wt_obj$new_subclusters_x)

In [ ]:
Idents(wt_obj) <- wt_obj$new_subclusters_x

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)
DotPlot(wt_obj, features =  "Atf3", group.by = "new_subclusters")

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)
DotPlot(wt_obj, features =  "Zfp711", group.by = "new_subclusters")

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)
DotPlot(wt_obj, features =  "Bcl6b", group.by = "new_subclusters")

# Heatmaps for the expression

In [ ]:
library(pheatmap)

## in vitro (CMO)

In [ ]:
ko_genes <- c('Atf3')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Atf3" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Atf3_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Atf3_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Zfp711')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Zfp711" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Zfp711_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Zfp711_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Bcl6b')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Bcl6b" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Bcl6b_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Bcl6b_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

### paralogs

In [ ]:
# https://www.flyrnai.org/tools/paralogs/web/taxid/10090 (paralog and/ DIOPT score >=3)
#https://www.genecards.org/#paralogs 

In [ ]:
ko_genes <- c('Atf3','Jdp2','Batf3','Batf','Fosl1','Fosb','Fosl2')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Atf3 and Atf3 Paralogs" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Atf3_and_Atf3_paralogs_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Atf3_and_Atf3_paralogs_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Zfp711','Zfy1','Zfy2','Zfx')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Zfp711 and Zfp711 Paralogs" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Zfp711_and_Zfp711_paralogs_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Zfp711_and_Zfp711_paralogs_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Bcl6b','Bcl6')

In [ ]:
avg_ko <- AverageExpression(wt_obj,features = ko_genes, layer = 'counts',group.by =  'new_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vitro Average Expression of Zfp711 and Zfp711 Paralogs" ,  # Add a title
)

In [ ]:

pdf("./figures_2/average_expression_Bcl6b_and_Bcl6b_paralogs_on_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Bcl6b_and_Bcl6b_paralogs_on_subclusters.png",width = 6.5, height = 4,units = 'in',res = 300)
print(x)
dev.off()   

## in vivo (Atlas)

In [ ]:
atlas

In [ ]:
ko_genes <- c('Atf3')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Atf3" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Atf3_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Atf3_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Zfp711')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Zfp711" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Zfp711_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Zfp711_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Bcl6b')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Bcl6b" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Bcl6b_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Bcl6b_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

### paralogs

In [ ]:
ko_genes <- c('Atf3','Jdp2','Batf3','Batf','Fosl1','Fosb','Fosl2')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Atf3 and Atf3 Paralogs" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Atf3_and_Atf3_paralogs_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Atf3_and_Atf3_paralogs_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Zfp711','Zfy1','Zfy2','Zfx')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Zfp711 and Zfp711 Paralogs" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Zfp711_and_Zfp711_paralogs_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Zfp711_and_Zfp711_paralogs_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

In [ ]:
ko_genes <- c('Bcl6b','Bcl6')

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Average Expression of Bcl6b and Bcl6 Paralogs" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_Bcl6b_and_Bcl6_paralogs_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
png("./figures_2/average_expression_Bcl6b_and_Bcl6_paralogs_atlas.png",width = 14, height = 5,units = 'in',res = 300)
print(x)
dev.off()   

# Stages, Somite and Anatomy (Atlas-in vivo)

In [ ]:
# Specify the desired order of the levels
stage_levels <- c(
  "E6.5", "E7.0", "E7.25", "E7.5", "E7.75", "E8.0", "E8.25", "E8.5",
  "E8.75", "E9.0", "E9.25", "E9.5", "Mixed gastrulation"
)

In [ ]:
atlas$stage_lv <-factor(atlas$stage, levels = stage_levels)

# Check that the levels are correct
table(atlas$stage_lv, useNA = "always")


In [ ]:
# Specify the desired order (spelling corrected and capitalized)
anatomy_levels <- c(
  "Pooled", 
  "YS", 
  "EP", 
  "Anterior", 
  "Posterior", 
  "Anterior section", 
  "Medial section", 
  "Posterior section"
)

# Create new factor column with ordered levels
atlas$anatomy_lv <- factor(
  atlas$anatomy,
  levels = anatomy_levels
)

# Check the result
table(atlas$anatomy_lv , useNA = "always")


In [ ]:
table(atlas$somite_count)

In [ ]:
# Specify the desired level order (numbers as characters, pooled last)
somite_levels <- c(
  "6", "7", "8", "10", "12","13", "14", "15", "16", "18", 
  "20", "21", "22", "23", "24", "Pooled"
)

# Create new factor column with ordered levels
atlas$somite_count_lv <- factor(
  atlas$somite_count,
  levels = somite_levels
)

# Check the result
table(atlas$somite_count_lv, useNA = "always")


In [ ]:
ko_genes <- "Atf3"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'stage_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Atf3 Average Expression\n Embryonic Stage" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Atf3_embryonic_stage.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Atf3_embryonic_stage.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Bcl6b"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'stage_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Bcl6b Average Expression\n Embryonic Stage" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Bcl6b_embryonic_stage.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Bcl6b_embryonic_stage.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Zfp711"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'stage_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Zfp711 Average Expression\n Embryonic Stage" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Zfp711_embryonic_stage.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Zfp711_embryonic_stage.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Atf3"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'anatomy_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Atf3 Average Expression\n Anatomic Location" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Atf3_anatomy.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Atf3_anatomy.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Zfp711"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'anatomy_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Zfp711 Average Expression\n Anatomic Location" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Zfp711_anatomy.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Zfp711_anatomy.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Bcl6b"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'anatomy_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Bcl6b Average Expression\n Anatomic Location" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Bcl6b_anatomy.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Bcl6b_anatomy.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Atf3"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'somite_count_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Atf3 Average Expression\n Somite Count" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Atf3_somite_count.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Atf3_count.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Zfp711"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'somite_count_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Zfp711 Average Expression\n Somite Count" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Zfp711_somite_count.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Zfp711_count.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


In [ ]:
ko_genes <- "Bcl6b"

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'somite_count_lv' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In vivo Bcl6b Average Expression\n Somite Count" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_2/average_expression_atlas_Bcl6b_somite_count.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:

png("./figures_2/average_expression_atlas_Bcl6b_count.png",width = 6.4, height = 5,res = 300, units = 'in')
print(x)
dev.off()   


#  Pseudotime

## Trying everything in one go (FAILES)

In [ ]:
atlas

In [ ]:
Idents(atlas) <- "celltype_extended_atlas"

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
endodermal_lineages <- c(
  "ExE endoderm",
  "Foregut",
  "Gut tube",
  "Hindgut",
  "Parietal endoderm",
  "Pharyngeal endoderm",
  "Thyroid primordium",
  "Visceral endoderm"
)

ectodermal_lineages <- c(
  "Amniotic ectoderm",
  "Branchial arch neural crest",
  "Dorsal hindbrain progenitors",
  "Dorsal midbrain neurons",
  "Dorsal spinal cord progenitors",
  "Early dorsal forebrain progenitors",
  "Ectoderm",
  "Epidermis",
  "ExE ectoderm",
  "Hindbrain floor plate",
  "Hindbrain neural progenitors",
  "Late dorsal forebrain progenitors",
  "Limb ectoderm",
  "Midbrain progenitors",
  "Midbrain/Hindbrain boundary",
  "Migratory neural crest",
  "Neural tube",
  "Non-neural ectoderm",
  "Optic vesicle",
  "Otic neural progenitors",
  "Otic placode",
  "Placodal ectoderm",
  "Spinal cord progenitors",
  "Surface ectoderm",
  "Ventral forebrain progenitors",
  "Ventral hindbrain progenitors"
)

In [ ]:
others <- c(
    "Anterior Primitive Streak",
    "Caudal epiblast",
    "Caudal mesoderm",
    "Cranial mesoderm",
    "Frontonasal mesenchyme",
    "Midgut",
    "NMPs",
    "NMPs/Mesoderm-biased",
    "Node",
    "Notochord",
    "PGC",
    "YS mesothelium-derived endothelial progenitors",
    "Chorioallantoic-derived erythroid progenitors"
    )
    

In [ ]:
endo_ecto <- c(endodermal_lineages,ectodermal_lineages,others)

In [ ]:
df <- subset(atlas, subset =  celltype_extended_atlas %in% endo_ecto, invert = TRUE )
df

In [ ]:
table(df$celltype_extended_atlas)

In [ ]:
mean(table(df$celltype_extended_atlas))

In [ ]:
median(table(df$celltype_extended_atlas))

In [ ]:
dim(df)

In [ ]:
dfx <- subset(df,downsample = 3475  )

In [ ]:
Idents(df) <- "celltype_extended_atlas"

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10 )
DimPlot(df, reduction = 'UMAP')+NoLegend()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10 )
DimPlot(dfx, reduction = 'UMAP')+NoLegend()

In [ ]:
source("../step11_slingshot_trails/slingshot_utils.R")

In [ ]:
suppressPackageStartupMessages({
  library(Seurat)
  library(plotly)
  options(rgl.printRglwidget = TRUE)
  library(Matrix)
  library(sparseMatrixStats)
  library(slingshot)
  library(tradeSeq)
  library(patchwork)
})


In [ ]:
df <- RunUMAP(df, reduction = "PCA",dims = 1:75, reduction.name = 'df_umap_2d')
dfx <- RunUMAP(dfx, reduction = "PCA",dims = 1:75, reduction.name = 'dfx_umap_2d')
df <- RunUMAP(df, reduction = "PCA",dims = 1:75,n.components = 3L, reduction.name = 'df_umap_3d')
dfx <- RunUMAP(dfx, reduction = "PCA",dims = 1:75,n.components = 3L, reduction.name = 'dfx_umap_3d')

In [ ]:
set.seed(1)
lineages_df_old_umap <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$UMAP@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_df_umap_2d <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$df_umap_2d@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_df_umap_3d <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$df_umap_3d@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
set.seed(1)
lineages_dfx_old_umap <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$UMAP@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_dfx_umap_2d <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$dfx_umap_2d@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_dfx_umap_3d <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$dfx_umap_3d@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
set.seed(1)
lineages_df_old_umap_mnn <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$UMAP@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_df_umap_2d_mnn <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$df_umap_2d@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_df_umap_3d_mnn <- as.SlingshotDataSet(getLineages(
  data           = df@reductions$df_umap_3d@cell.embeddings,
  clusterLabels  = df$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
set.seed(1)
lineages_dfx_old_umap_mnn <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$UMAP@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_dfx_umap_2d_mnn <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$dfx_umap_2d@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

set.seed(1)
lineages_dfx_umap_3d_mnn <- as.SlingshotDataSet(getLineages(
  data           = dfx@reductions$dfx_umap_3d@cell.embeddings,
  clusterLabels  = dfx$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 # end.clus       = ENDS, # You can also define the ENDS!
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_df_old_umap <- as.SlingshotDataSet(getCurves(data = lineages_df_old_umap))

In [ ]:
curves_df_umap_2d <- as.SlingshotDataSet(getCurves(data = lineages_df_umap_2d))

In [ ]:
curves_df_umap_3d <- as.SlingshotDataSet(getCurves(data = lineages_df_umap_3d))

In [ ]:
curves_df_old_umap_mnn <- as.SlingshotDataSet(getCurves(data = lineages_df_old_umap_mnn))

In [ ]:
curves_df_umap_2d_mnn <- as.SlingshotDataSet(getCurves(data = lineages_df_umap_2d_mnn))

In [ ]:
curves_df_umap_3d_mnn <- as.SlingshotDataSet(getCurves(data = lineages_df_umap_3d_mnn))

In [ ]:
# Define curves
curves_dfx_old_umap <- as.SlingshotDataSet(getCurves(data = lineages_dfx_old_umap))

In [ ]:
curves_dfx_umap_2d <- as.SlingshotDataSet(getCurves(data = lineages_dfx_umap_2d))

In [ ]:
curves_dfx_umap_3d <- as.SlingshotDataSet(getCurves(data = lineages_dfx_umap_3d))

In [ ]:
curves_dfx_old_umap_mnn <- as.SlingshotDataSet(getCurves(data = lineages_dfx_old_umap_mnn))


In [ ]:
curves_dfx_umap_2d_mnn <- as.SlingshotDataSet(getCurves(data = lineages_dfx_umap_2d_mnn))

In [ ]:
curves_dfx_umap_3d_mnn <- as.SlingshotDataSet(getCurves(data = lineages_dfx_umap_3d_mnn))

In [ ]:
ls()

In [ ]:
plot_umap_with_curves

In [ ]:
markers_emp_tf <- markers_emp[rownames(markers_emp) %in% tf_list$Symbol,]

In [ ]:
df$cell_type_subclusters <- df$celltype_extended_atlas

In [ ]:
df@meta.data$seurat_clusters <- df$celltype_extended_atlas

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)

# Extract pseudotime for Lineage1 from curves object
pseudotime_lineage1 <- slingPseudotime(curves_df_old_umap, na = FALSE)[, "Lineage1"]

# Get ATF3 expression from your Seurat object
# Assuming your Seurat object is named 'seurat_obj' - adjust if different
atf3_expression <- GetAssayData(df, slot = "data")["Atf3", ]

# Create a data frame for plotting (including zero expression)
plot_data <- data.frame(
  pseudotime = pseudotime_lineage1,
  expression = atf3_expression[names(pseudotime_lineage1)]
)

# Remove only NA values (keep zeros)
plot_data <- plot_data[complete.cases(plot_data), ]

# Create the trend-only plot
p <- ggplot(plot_data, aes(x = pseudotime, y = expression)) +
  geom_smooth(method = "gam", formula = y ~ s(x, bs = "cs"), 
              se = TRUE, color = "red", fill = "lightcoral", 
              linewidth = 1.2, alpha = 0.3) +
  labs(
    title = "ATF3 Expression Trend Along Lineage1",
    x = "Pseudotime",
    y = "ATF3 Expression",
    subtitle = paste0("Trend line based on ", nrow(plot_data), " cells (including zero expression)")
  ) +
  theme_classic() +
  theme(
    plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
    plot.subtitle = element_text(size = 12, hjust = 0.5),
    axis.title = element_text(size = 12),
    axis.text = element_text(size = 10),
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
  )

# Display the plot
print(p)

# Optional: Save the plot
# ggsave("ATF3_Lineage1_trend.pdf", plot = p, width = 8, height = 6)

In [ ]:
seurat_obj = df

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)
library(dplyr)

# Extract pseudotime for Lineage1 from curves object
pseudotime_lineage1 <- slingPseudotime(curves_df_old_umap, na = FALSE)[, "Lineage2"]

# Get ATF3 expression from your Seurat object
# Assuming your Seurat object is named 'seurat_obj' - adjust if different
atf3_expression <- GetAssayData(seurat_obj, slot = "data")["Atf3", ]

# Get cell type information from seurat_clusters
cell_types <- seurat_obj$seurat_clusters[names(pseudotime_lineage1)]

# Create a data frame for plotting (including zero expression)
plot_data <- data.frame(
  pseudotime = pseudotime_lineage1,
  expression = atf3_expression[names(pseudotime_lineage1)],
  cell_type = cell_types
)

# Remove only NA values (keep zeros)
plot_data <- plot_data[complete.cases(plot_data), ]

# Count cells per cell type and group small cell types
cell_type_counts <- table(plot_data$cell_type)
small_cell_types <- names(cell_type_counts[cell_type_counts < 100])

# Create a new column for plotting - group small cell types as "Other"
plot_data$cell_type_plot <- as.character(plot_data$cell_type)
plot_data$cell_type_plot[plot_data$cell_type %in% small_cell_types] <- "Other (< 50 cells)"

# Order cell types by their median pseudotime
cell_type_order <- plot_data %>%
  group_by(cell_type_plot) %>%
  summarise(median_pseudotime = median(pseudotime, na.rm = TRUE)) %>%
  arrange(median_pseudotime) %>%
  pull(cell_type_plot)

plot_data$cell_type_plot <- factor(plot_data$cell_type_plot, levels = cell_type_order)

# Create pseudotime bins to determine dominant cell type in each region
n_bins <- 50  # Number of bins along pseudotime
pseudotime_range <- range(plot_data$pseudotime, na.rm = TRUE)
bin_breaks <- seq(pseudotime_range[1], pseudotime_range[2], length.out = n_bins + 1)
plot_data$pseudotime_bin <- cut(plot_data$pseudotime, breaks = bin_breaks, include.lowest = TRUE)

# Find dominant cell type in each bin (excluding small cell types)
dominant_celltype <- plot_data %>%
  filter(cell_type_plot != "Other (< 50 cells)") %>%  # Exclude small cell types
  group_by(pseudotime_bin) %>%
  count(cell_type_plot) %>%
  slice_max(n, n = 1, with_ties = FALSE) %>%
  ungroup() %>%
  select(pseudotime_bin, dominant_type = cell_type_plot)

# Create bin centers for plotting
bin_centers <- (bin_breaks[-1] + bin_breaks[-length(bin_breaks)]) / 2
dominant_celltype$pseudotime_center <- bin_centers[as.numeric(dominant_celltype$pseudotime_bin)]

# Fit GAM model to get smooth predictions
gam_model <- gam(expression ~ s(pseudotime, bs = "cs"), data = plot_data)
smooth_data <- data.frame(
  pseudotime = bin_centers
)
smooth_data$expression <- predict(gam_model, newdata = smooth_data)

# Merge with dominant cell types
smooth_data <- merge(smooth_data, 
                     dominant_celltype[, c("pseudotime_center", "dominant_type")], 
                     by.x = "pseudotime", by.y = "pseudotime_center", all.x = TRUE)

# Create segment groups to break lines when cell type changes or has gaps
smooth_data <- smooth_data[order(smooth_data$pseudotime), ]
smooth_data$segment_group <- cumsum(c(TRUE, smooth_data$dominant_type[-1] != smooth_data$dominant_type[-nrow(smooth_data)] | 
                                     is.na(smooth_data$dominant_type[-1]) | is.na(smooth_data$dominant_type[-nrow(smooth_data)])))

# Remove rows with NA dominant_type (bins with no major cell types)
smooth_data <- smooth_data[!is.na(smooth_data$dominant_type), ]

# Set up colors (only for major cell types)
major_cell_types <- levels(plot_data$cell_type_plot)[levels(plot_data$cell_type_plot) != "Other (< 50 cells)"]
colors <- character(length(major_cell_types))
names(colors) <- major_cell_types

# Assign colors only to major cell types
if(exists("r3dcol") && !is.null(r3dcol$cols_94) && length(r3dcol$cols_94) > 0) {
  colors[major_cell_types] <- r3dcol$cols_94[1:length(major_cell_types)]
} else {
  colors[major_cell_types] <- rainbow(length(major_cell_types))
}

# Create the trend plot with cell-type-colored segments
p <- ggplot() +
  # Add confidence interval using original data
  geom_smooth(data = plot_data, aes(x = pseudotime, y = expression),
              method = "gam", formula = y ~ s(x, bs = "cs"), 
              se = TRUE, color = NA, fill = "darkgrey", alpha = 0.5) +
  # Add colored line segments based on dominant cell type (broken into segments)
  geom_path(data = smooth_data, aes(x = pseudotime, y = expression, 
                                    color = dominant_type, group = segment_group), 
            linewidth = 2) +
  scale_color_manual(values = colors, name = "Dominant Cell Type") +
  labs(
    title = "ATF3 Expression Trend Along Lineage2",
    x = "Pseudotime",
    y = "ATF3 Expression",
    subtitle = paste0("Line colored by dominant cell type in each pseudotime region (n = ", nrow(plot_data), " cells)")
  ) +
  theme_classic() +
  theme(
    plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
    plot.subtitle = element_text(size = 11, hjust = 0.5),
    axis.title = element_text(size = 12),
    axis.text = element_text(size = 10),
    legend.title = element_text(size = 11),
    legend.text = element_text(size = 9),
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
  ) +
  guides(color = guide_legend(override.aes = list(linewidth = 3)))

# Display the plot
print(p)

# Optional: Save the plot
# ggsave("ATF3_Lineage1_trend.pdf", plot = p, width = 8, height = 6)

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)
library(dplyr)

#' Diagnostic function to check pseudotime data
#'
#' @param seurat_obj Seurat object
#' @param curves_obj Slingshot curves object
#' @param cell_type_column Character. Column name for cell types
#'
check_pseudotime_data <- function(seurat_obj, curves_obj, cell_type_column = "seurat_clusters") {
  
  cat("=== PSEUDOTIME DATA DIAGNOSTIC ===\n")
  
  # Check available lineages
  lineages <- colnames(slingPseudotime(curves_obj))
  cat("Available lineages:", paste(lineages, collapse = ", "), "\n")
  
  # Check pseudotime data for each lineage
  pseudotime_matrix <- slingPseudotime(curves_obj, na = FALSE)
  
  for (lineage in lineages) {
    pseudotime_values <- pseudotime_matrix[, lineage]
    n_total <- length(pseudotime_values)
    n_finite <- sum(is.finite(pseudotime_values))
    n_na <- sum(is.na(pseudotime_values))
    
    cat("\n", lineage, ":\n")
    cat("  Total cells:", n_total, "\n")
    cat("  Finite values:", n_finite, "\n")
    cat("  NA values:", n_na, "\n")
    
    if (n_finite > 0) {
      cat("  Range:", round(range(pseudotime_values, na.rm = TRUE), 3), "\n")
    }
  }
  
  # Check cell name overlap
  curve_cells <- rownames(pseudotime_matrix)
  seurat_cells <- colnames(seurat_obj)
  overlap <- intersect(curve_cells, seurat_cells)
  
  cat("\n=== CELL NAME OVERLAP ===\n")
  cat("Cells in curves object:", length(curve_cells), "\n")
  cat("Cells in Seurat object:", length(seurat_cells), "\n")
  cat("Overlapping cells:", length(overlap), "\n")
  
  if (length(overlap) == 0) {
    cat("WARNING: No overlapping cell names found!\n")
    cat("First 5 curve cell names:", head(curve_cells, 5), "\n")
    cat("First 5 Seurat cell names:", head(seurat_cells, 5), "\n")
  }
  
  return(invisible(NULL))
}

#' Plot Gene Expression Along Pseudotime with Cell Type Coloring
#'
#' This function creates a trend plot showing gene expression along a specific lineage
#' pseudotime trajectory, with the trend line colored by dominant cell types in each region.
#'
#' @param seurat_obj Seurat object containing gene expression data
#' @param curves_obj Slingshot curves object containing pseudotime information
#' @param gene_name Character. Name of the gene to plot (must match gene names in Seurat object)
#' @param lineage_name Character. Name of the lineage to plot (e.g., "Lineage1", "Lineage2")
#' @param min_cells Numeric. Minimum number of cells required for a cell type to be considered major (default: 50)
#' @param n_bins Numeric. Number of bins to divide pseudotime axis for determining dominant cell types (default: 50)
#' @param color_palette Character vector. Colors to use for major cell types. If NULL, uses rainbow colors (default: NULL)
#' @param cell_type_column Character. Column name in Seurat object metadata containing cell type information (default: "seurat_clusters")
#' @param expression_slot Character. Which slot to use for gene expression ("data", "counts", or "scale.data") (default: "data")
#' @param save_plot Logical. Whether to save the plot (default: FALSE)
#' @param filename Character. Filename for saving plot (default: NULL)
#' @param plot_width Numeric. Width for saved plot (default: 10)
#' @param plot_height Numeric. Height for saved plot (default: 6)
#'
#' @return ggplot object
#'
plot_gene_pseudotime <- function(seurat_obj,
                                curves_obj,
                                gene_name,
                                lineage_name,
                                min_cells = 50,
                                n_bins = 50,
                                color_palette = NULL,
                                cell_type_column = "seurat_clusters",
                                expression_slot = "data",
                                save_plot = FALSE,
                                filename = NULL,
                                plot_width = 10,
                                plot_height = 6) {
  
  # Input validation
  message("=== STARTING INPUT VALIDATION ===")
  
  if (!gene_name %in% rownames(seurat_obj)) {
    stop(paste("Gene", gene_name, "not found in Seurat object"))
  }
  message("✓ Gene found in Seurat object")
  
  if (!lineage_name %in% colnames(slingPseudotime(curves_obj))) {
    available_lineages <- colnames(slingPseudotime(curves_obj))
    stop(paste("Lineage", lineage_name, "not found. Available lineages:", paste(available_lineages, collapse = ", ")))
  }
  message("✓ Lineage found in curves object")
  
  if (!cell_type_column %in% colnames(seurat_obj@meta.data)) {
    stop(paste("Cell type column", cell_type_column, "not found in Seurat object metadata"))
  }
  message("✓ Cell type column found in metadata")
  
  message("=== EXTRACTING DATA ===")
  
  # Extract pseudotime for specified lineage
  pseudotime_matrix <- slingPseudotime(curves_obj)
  pseudotime_data <- pseudotime_matrix[, lineage_name]
  
  message(paste("Extracted pseudotime for", lineage_name, "- length:", length(pseudotime_data)))
  message(paste("Number of NA values:", sum(is.na(pseudotime_data))))
  message(paste("Number of finite values:", sum(is.finite(pseudotime_data))))
  if(sum(is.finite(pseudotime_data)) > 0) {
    message(paste("Pseudotime range:", paste(range(pseudotime_data, na.rm = TRUE), collapse = " to ")))
  }
  
  # Get gene expression
  gene_expression <- GetAssayData(seurat_obj, slot = expression_slot)[gene_name, ]
  message(paste("Extracted expression for", gene_name, "- length:", length(gene_expression)))
  
  # Get cell type information
  cell_types <- seurat_obj@meta.data[names(pseudotime_data), cell_type_column]
  message(paste("Extracted cell types - length:", length(cell_types)))
  message(paste("Number of NA cell types:", sum(is.na(cell_types))))
  message(paste("First few cell types:", paste(head(cell_types, na.rm = TRUE), collapse = ", ")))
  
  # Create data frame
  plot_data <- data.frame(
    pseudotime = pseudotime_data,
    expression = gene_expression[names(pseudotime_data)],
    cell_type = cell_types
  )
  
  message(paste("Created plot_data with", nrow(plot_data), "rows"))
  message(paste("Complete cases before filtering:", sum(complete.cases(plot_data))))
  
  # Remove NA values
  plot_data <- plot_data[complete.cases(plot_data), ]
  
  message(paste("Complete cases after filtering:", nrow(plot_data)))
  
  # Check if we have any valid data after removing NAs
  if (nrow(plot_data) == 0) {
    message("=== DEBUGGING INFO ===")
    message("Original pseudotime_data length:", length(pseudotime_data))
    message("Original gene_expression length:", length(gene_expression))
    message("Original cell_types length:", length(cell_types))
    message("Names match between pseudotime and expression:", 
            identical(names(pseudotime_data), names(gene_expression[names(pseudotime_data)])))
    message("Sample of pseudotime names:", paste(head(names(pseudotime_data)), collapse = ", "))
    message("Sample of expression names:", paste(head(names(gene_expression)), collapse = ", "))
    stop(paste("No valid data found for", lineage_name, ". Check if the lineage exists and has valid pseudotime values."))
  }
  
  # Check if pseudotime values are finite
  if (!all(is.finite(plot_data$pseudotime))) {
    stop("Non-finite pseudotime values found. Please check your slingshot curves object.")
  }
  
  message(paste("Found", nrow(plot_data), "cells with valid pseudotime and expression data for", lineage_name))
  
  # Count cells per cell type and identify small cell types
  cell_type_counts <- table(plot_data$cell_type)
  small_cell_types <- names(cell_type_counts[cell_type_counts < min_cells])
  
  message(paste("Cell type counts:"))
  print(cell_type_counts)
  message(paste("Small cell types (< ", min_cells, " cells):", paste(small_cell_types, collapse = ", ")))
  
  # Group small cell types
  plot_data$cell_type_plot <- as.character(plot_data$cell_type)
  plot_data$cell_type_plot[plot_data$cell_type %in% small_cell_types] <- paste0("Other (< ", min_cells, " cells)")
  
  # Order cell types by median pseudotime
  cell_type_order <- plot_data %>%
    group_by(cell_type_plot) %>%
    summarise(median_pseudotime = median(pseudotime, na.rm = TRUE), .groups = 'drop') %>%
    arrange(median_pseudotime) %>%
    pull(cell_type_plot)
  
  plot_data$cell_type_plot <- factor(plot_data$cell_type_plot, levels = cell_type_order)
  
  # Create pseudotime bins
  pseudotime_range <- range(plot_data$pseudotime, na.rm = TRUE)
  bin_breaks <- seq(pseudotime_range[1], pseudotime_range[2], length.out = n_bins + 1)
  plot_data$pseudotime_bin <- cut(plot_data$pseudotime, breaks = bin_breaks, include.lowest = TRUE)
  
  # Find dominant cell type in each bin (excluding small cell types)
  dominant_celltype <- plot_data %>%
    filter(cell_type_plot != paste0("Other (< ", min_cells, " cells)")) %>%
    group_by(pseudotime_bin) %>%
    count(cell_type_plot) %>%
    slice_max(n, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(pseudotime_bin, dominant_type = cell_type_plot)
  
  # Create bin centers
  bin_centers <- (bin_breaks[-1] + bin_breaks[-length(bin_breaks)]) / 2
  dominant_celltype$pseudotime_center <- bin_centers[as.numeric(dominant_celltype$pseudotime_bin)]
  
  # Fit GAM model for smooth predictions
  gam_model <- gam(expression ~ s(pseudotime, bs = "cs"), data = plot_data)
  smooth_data <- data.frame(pseudotime = bin_centers)
  smooth_data$expression <- predict(gam_model, newdata = smooth_data)
  
  # Merge with dominant cell types
  smooth_data <- merge(smooth_data, 
                       dominant_celltype[, c("pseudotime_center", "dominant_type")], 
                       by.x = "pseudotime", by.y = "pseudotime_center", all.x = TRUE)
  
  # Create segment groups to break lines
  smooth_data <- smooth_data[order(smooth_data$pseudotime), ]
  smooth_data$segment_group <- cumsum(c(TRUE, smooth_data$dominant_type[-1] != smooth_data$dominant_type[-nrow(smooth_data)] | 
                                       is.na(smooth_data$dominant_type[-1]) | is.na(smooth_data$dominant_type[-nrow(smooth_data)])))
  
  # Remove rows with NA dominant_type
  smooth_data <- smooth_data[!is.na(smooth_data$dominant_type), ]
  
  # Set up colors
  major_cell_types <- levels(plot_data$cell_type_plot)[levels(plot_data$cell_type_plot) != paste0("Other (< ", min_cells, " cells)")]
  
  if (is.null(color_palette)) {
    colors <- rainbow(length(major_cell_types))
  } else {
    if (length(color_palette) < length(major_cell_types)) {
      warning("Not enough colors in palette. Recycling colors.")
      colors <- rep(color_palette, length.out = length(major_cell_types))
    } else {
      colors <- color_palette[1:length(major_cell_types)]
    }
  }
  names(colors) <- major_cell_types
  
  # Create plot
  p <- ggplot() +
    # Add confidence interval
    geom_smooth(data = plot_data, aes(x = pseudotime, y = expression),
                method = "gam", formula = y ~ s(x, bs = "cs"), 
                se = TRUE, color = NA, fill = "darkgrey", alpha = 0.5) +
    # Add colored line segments
    geom_path(data = smooth_data, aes(x = pseudotime, y = expression, 
                                      color = dominant_type, group = segment_group), 
              linewidth = 2) +
    scale_color_manual(values = colors, name = "Dominant Cell Type") +
    labs(
      title = paste(gene_name, "Expression Trend Along", lineage_name),
      x = "Pseudotime",
      y = paste(gene_name, "Expression"),
      subtitle = paste0("Line colored by dominant cell type in each pseudotime region (n = ", 
                       nrow(plot_data), " cells, min_cells = ", min_cells, ", bins = ", n_bins, ")")
    ) +
    theme_classic() +
    theme(
      plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 11, hjust = 0.5),
      axis.title = element_text(size = 12),
      axis.text = element_text(size = 10),
      legend.title = element_text(size = 11),
      legend.text = element_text(size = 9),
      panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
    ) +
    guides(color = guide_legend(override.aes = list(linewidth = 3)))
  
  # Save plot if requested
  if (save_plot) {
    if (is.null(filename)) {
      filename <- paste0(gene_name, "_", lineage_name, "_pseudotime_trend.pdf")
    }
    ggsave(filename, plot = p, width = plot_width, height = plot_height)
    message(paste("Plot saved as:", filename))
  }
  
  return(p)
}

# Example usage:
# p <- plot_gene_pseudotime(seurat_obj, curves_df_old_umap, "Atf3", "Lineage1")

In [ ]:
p <- plot_gene_pseudotime(
   seurat_obj = df,
   curves_obj = curves_df_old_umap,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 100,
   color_palette = r3dcol$cols_94,
   cell_type_column = "seurat_clusters"
)
options(repr.plot.width = 10, repr.plot.height = 5)
print(p)

In [ ]:
curves_df_old_umap

In [ ]:
curves_df_old_umap@lineages$Lineage6

In [ ]:
curves_df_old_umap@lineages$Lineage5

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
plot_umap_pseudotime_and_centroids(
  seurat_obj = df,
  curves = curves,
  embedding = "UMAP",
 cluster_col = "orig.ident",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 4
)


In [ ]:
df$cell_type_subclusters <- df$celltype_extended_atlas

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "dfx_umap_3d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_umap_3d_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_umap_3d_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "dfx_umap_2d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_umap_2d_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_umap_2d_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "UMAP",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_old_umap_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_old_umap_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "dfx_umap_3d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_umap_3d,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_umap_3d

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "dfx_umap_2d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_umap_2d,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_umap_2d

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = dfx,
  reduction = "UMAP",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_dfx_old_umap,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_dfx_old_umap

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "df_umap_3d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_umap_3d_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_umap_3d_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "df_umap_2d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_umap_2d_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_umap_2d_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "UMAP",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_old_umap_mnn,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_old_umap_mnn

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "df_umap_3d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_umap_3d,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_umap_3d

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "df_umap_2d",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_umap_2d,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_umap_2d

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
plot_umap_with_curves(seurat_obj = df,
  reduction = "UMAP",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_df_old_umap,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)

In [ ]:
curves_df_old_umap

### Notes

It didn't work, I will try to do it more lineage specified mannar.

In [ ]:
ls()

In [ ]:
rm(lm_obj)

In [ ]:
rm(wt_obj)

In [ ]:
ls()

In [ ]:
gc()

## In vitro (CMO)

In [ ]:
obj

In [ ]:
table(obj$new_subclusters_x)

### Late Mesoderm

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','Epicardium','Mesenchyme','Limb Mesoderm','Allantois'))

#### Epicardium

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','Epicardium'))

In [ ]:
xobj

In [ ]:
table(obj$Conditions_lv)

In [ ]:
xobj <- subset(xobj, subset = Conditions_lv %in% "Control")
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters', label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap0@cell.embeddings,
  clusterLabels  = xobj$new_subclusters,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c("Epicardium 3"),
  start.clus     = "Naive Pluripotency"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap0",
  cluster_col = "new_subclusters",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap0",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
curves_xobj

In [ ]:
p_epicardium <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_epicardium)

In [ ]:
saveRDS(xobj, "./cmo_v3_latemesoderm_epicardium_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './cmo_v3_latemesoderm_epicardium_slingshot_lin.rds')
saveRDS(curves_xobj, './cmo_v3_latemesoderm_epicardium_slingshot_curves.rds')

In [ ]:
cmo_lm_epicardium_xobj <-xobj
lineages_cmo_lm_epicardium <-lineages_xobj
curves_cmo_lm_epicardium <- curves_xobj

#### Limb Mesoderm

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','Limb Mesoderm'))

In [ ]:
xobj

In [ ]:
table(xobj$new_subclusters_x)

In [ ]:
table(obj$Conditions_lv)

In [ ]:
xobj <- subset(xobj, subset = Conditions_lv %in% "Control")
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters', label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap0@cell.embeddings,
  clusterLabels  = xobj$new_subclusters,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c("Limb Mesoderm 2"),
  start.clus     = "Naive Pluripotency"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap0",
  cluster_col = "new_subclusters",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap0",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
curves_xobj

In [ ]:
p_limb_mesoderm <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_limb_mesoderm)

In [ ]:
saveRDS(xobj, "./cmo_v3_latemesoderm_limb_mesoderm_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './cmo_v3_latemesoderm_limb_mesoderm_slingshot_lin.rds')
saveRDS(curves_xobj, './cmo_v3_latemesoderm_limb_mesoderm_slingshot_curves.rds')

In [ ]:
cmo_lm_limb_mesoderm_xobj <-xobj
lineages_cmo_lm_limb_mesoderm <-lineages_xobj
curves_cmo_lm_limb_mesoderm <- curves_xobj

#### Mesenchyme

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','Mesenchyme'))

In [ ]:
xobj

In [ ]:
table(xobj$new_subclusters_x)

In [ ]:
table(obj$Conditions_lv)

In [ ]:
xobj <- subset(xobj, subset = Conditions_lv %in% "Control")
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters', label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap0@cell.embeddings,
  clusterLabels  = xobj$new_subclusters,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c("Mesenchyme 2"),
  start.clus     = "Naive Pluripotency"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap0",
  cluster_col = "new_subclusters",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap0",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
curves_xobj

In [ ]:
p_mesenchyme <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_mesenchyme)

In [ ]:
saveRDS(xobj, "./cmo_v3_latemesoderm_mesenchyme_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './cmo_v3_latemesoderm_mesenchyme_slingshot_lin.rds')
saveRDS(curves_xobj, './cmo_v3_latemesoderm_mesenchyme_slingshot_curves.rds')

In [ ]:
cmo_lm_mesenchyme_xobj <-xobj
lineages_cmo_lm_mesenchyme <-lineages_xobj
curves_cmo_lm_mesenchyme <- curves_xobj

#### Allantois

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','Allantois', 'Epicardium', 'Mesenchyme'))

In [ ]:
xobj

In [ ]:
table(xobj$new_subclusters_x)

In [ ]:
table(obj$Conditions_lv)

In [ ]:
xobj <- subset(xobj, subset = Conditions_lv %in% "Control")
xobj

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'new_subclusters')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'new_subclusters')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'new_subclusters')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters', label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap3@cell.embeddings,
  clusterLabels  = xobj$new_subclusters,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
 end.clus       = c("Allantois 4"),
  start.clus     = "Naive Pluripotency"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap3",
  cluster_col = "new_subclusters",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap3",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
curves_xobj

In [ ]:
p_allantois <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_allantois)

In [ ]:
saveRDS(xobj, "./cmo_v3_latemesoderm_allantois_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './cmo_v3_latemesoderm_allantois_slingshot_lin.rds')
saveRDS(curves_xobj, './cmo_v3_latemesoderm_allantois_slingshot_curves.rds')

In [ ]:
cmo_lm_allantois_xobj <-xobj
lineages_cmo_lm_allantois <-lineages_xobj
curves_cmo_lm_allantois <- curves_xobj

### Bloood

In [ ]:
obj

In [ ]:
table(obj$new_subclusters_x)

In [ ]:
xobj <- subset(obj, subset = new_subclusters_x %in% c('Naive Pluripotency','Epiblast','Primitive Streak','Nascent Mesoderm','Lateral Plate Mesoderm','HEP/BP','Erythroid', 'MkP','MEP','Endothelium','Endocardium','EMP'))

In [ ]:
xobj

In [ ]:
table(obj$Conditions_lv)

In [ ]:
xobj <- subset(xobj, subset = Conditions_lv %in% "Control")
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# Findvariable Features
xobj <- FindVariableFeatures(object = xobj, nfeatures = 4000)

In [ ]:
xobj <- ScaleData(object = xobj, 
                          features = VariableFeatures(xobj),
                          vars.to.regress = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.ribo", "G2M.Score", "S.Score"),
                          block.size = 20000)

In [ ]:
xobj <- RunPCA(xobj,npcs = 50, reduction.name = "xobj_pca",
                       features =  VariableFeatures(xobj)
                      )

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'new_subclusters_x')

In [ ]:
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94,label = TRUE, group.by = 'new_subclusters')

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap3@cell.embeddings,
  clusterLabels  = xobj$new_subclusters,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c("Erythroid 4","MkP 2","MEP","Endothelium 2","Endocardium","EMP 4"),
  start.clus     = "Naive Pluripotency"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap3",
  cluster_col = "new_subclusters",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap3",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
curves_xobj

In [ ]:
p_emp <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_emp)

In [ ]:
p_mkp <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage2",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_mkp)

In [ ]:
p_erythroid <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage3",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_erythroid)

In [ ]:
p_endothelium <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage4",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_endothelium)

In [ ]:
p_MEP <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage5",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
p_MEP

In [ ]:
p_ery2<- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage6",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
p_ery2

In [ ]:
p_endocardium <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage7",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "new_subclusters_x"
)

In [ ]:
p_endocardium

In [ ]:
saveRDS(xobj, "./cmo_v3_HEP_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './cmo_v3_hep_slingshot_lin.rds')
saveRDS(curves_xobj, './cmo_v3_hep_slingshot_curves.rds')

In [ ]:
# after crush
cmo_hep_xobj <- readRDS('./cmo_v3_HEP_lineage.rds') # it should be v4 actually.
lineages_cmo_hep <- readRDS('./cmo_v3_hep_slingshot_lin.rds')
curves_cmo_hep <- readRDS('./cmo_v3_hep_slingshot_curves.rds')

In [ ]:
cmo_hep_xobj

In [ ]:
lineages_cmo_hep

In [ ]:
curves_cmo_hep

## In vivo (Atlas)

### Late Mesoderm (Epicardium, Mesenchyme, Allantois, Limb mesoderm)

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm','Lateral plate mesoderm','Epicardium','Allantois','Mesenchyme','Limb mesoderm'))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# Findvariable Features
xobj <- FindVariableFeatures(object = xobj, nfeatures = 4000)

In [ ]:
xobj <- ScaleData(object = xobj, 
                          features = VariableFeatures(xobj),
                          vars.to.regress = c("nFeature_originalexp", "nCount_originalexp", "G2M_score", "S_score"),
                          block.size = 20000)

In [ ]:
xobj <- RunPCA(xobj,npcs = 50, reduction.name = "xobj_pca",
                       features =  VariableFeatures(xobj)
                      )

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2")
xobj <- RunUMAP(xobj, reduction = "xobj_pca", dims = 1:50, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3")

In [ ]:
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('Limb mesoderm','Mesenchyme','Epicardium','Allantois'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap3",
 cluster_col = "new_subclusters",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)
library(dplyr)

#' Diagnostic function to check pseudotime data
#'
#' @param seurat_obj Seurat object
#' @param curves_obj Slingshot curves object
#' @param cell_type_column Character. Column name for cell types
#'
check_pseudotime_data <- function(seurat_obj, curves_obj, cell_type_column = "seurat_clusters") {
  
  cat("=== PSEUDOTIME DATA DIAGNOSTIC ===\n")
  
  # Check available lineages
  lineages <- colnames(slingPseudotime(curves_obj))
  cat("Available lineages:", paste(lineages, collapse = ", "), "\n")
  
  # Check pseudotime data for each lineage
  pseudotime_matrix <- slingPseudotime(curves_obj, na = FALSE)
  
  for (lineage in lineages) {
    pseudotime_values <- pseudotime_matrix[, lineage]
    n_total <- length(pseudotime_values)
    n_finite <- sum(is.finite(pseudotime_values))
    n_na <- sum(is.na(pseudotime_values))
    
    cat("\n", lineage, ":\n")
    cat("  Total cells:", n_total, "\n")
    cat("  Finite values:", n_finite, "\n")
    cat("  NA values:", n_na, "\n")
    
    if (n_finite > 0) {
      cat("  Range:", round(range(pseudotime_values, na.rm = TRUE), 3), "\n")
    }
  }
  
  # Check cell name overlap
  curve_cells <- rownames(pseudotime_matrix)
  seurat_cells <- colnames(seurat_obj)
  overlap <- intersect(curve_cells, seurat_cells)
  
  cat("\n=== CELL NAME OVERLAP ===\n")
  cat("Cells in curves object:", length(curve_cells), "\n")
  cat("Cells in Seurat object:", length(seurat_cells), "\n")
  cat("Overlapping cells:", length(overlap), "\n")
  
  if (length(overlap) == 0) {
    cat("WARNING: No overlapping cell names found!\n")
    cat("First 5 curve cell names:", head(curve_cells, 5), "\n")
    cat("First 5 Seurat cell names:", head(seurat_cells, 5), "\n")
  }
  
  return(invisible(NULL))
}

#' Plot Gene Expression Along Pseudotime with Cell Type Coloring
#'
#' This function creates a trend plot showing gene expression along a specific lineage
#' pseudotime trajectory, with the trend line colored by dominant cell types in each region.
#'
#' @param seurat_obj Seurat object containing gene expression data
#' @param curves_obj Slingshot curves object containing pseudotime information
#' @param gene_name Character. Name of the gene to plot (must match gene names in Seurat object)
#' @param lineage_name Character. Name of the lineage to plot (e.g., "Lineage1", "Lineage2")
#' @param min_cells Numeric. Minimum number of cells required for a cell type to be considered major (default: 50)
#' @param n_bins Numeric. Number of bins to divide pseudotime axis for determining dominant cell types (default: 50)
#' @param color_palette Character vector. Colors to use for major cell types. If NULL, uses rainbow colors (default: NULL)
#' @param cell_type_column Character. Column name in Seurat object metadata containing cell type information (default: "seurat_clusters")
#' @param expression_slot Character. Which slot to use for gene expression ("data", "counts", or "scale.data") (default: "data")
#' @param ylim Numeric vector of length 2. Y-axis limits c(min, max). If NULL, uses automatic limits (default: NULL)
#' @param lineage_destination Character. Optional destination/endpoint name for the lineage (e.g., "Erythroid", "Neural"). If provided, will be added to plot title (default: NULL)
#' @param save_plot Logical. Whether to save the plot (default: FALSE)
#' @param filename Character. Filename for saving plot (default: NULL)
#' @param plot_width Numeric. Width for saved plot (default: 10)
#' @param plot_height Numeric. Height for saved plot (default: 6)
#'
#' @return ggplot object
#'
plot_gene_pseudotime <- function(seurat_obj,
                                curves_obj,
                                gene_name,
                                lineage_name,
                                min_cells = 50,
                                n_bins = 50,
                                color_palette = NULL,
                                cell_type_column = "seurat_clusters",
                                expression_slot = "data",
                                ylim = NULL,
                                lineage_destination = NULL,
                                save_plot = FALSE,
                                filename = NULL,
                                plot_width = 10,
                                plot_height = 6) {
  
  # Input validation
  message("=== STARTING INPUT VALIDATION ===")
  
  if (!gene_name %in% rownames(seurat_obj)) {
    stop(paste("Gene", gene_name, "not found in Seurat object"))
  }
  message("✓ Gene found in Seurat object")
  
  if (!lineage_name %in% colnames(slingPseudotime(curves_obj))) {
    available_lineages <- colnames(slingPseudotime(curves_obj))
    stop(paste("Lineage", lineage_name, "not found. Available lineages:", paste(available_lineages, collapse = ", ")))
  }
  message("✓ Lineage found in curves object")
  
  if (!cell_type_column %in% colnames(seurat_obj@meta.data)) {
    stop(paste("Cell type column", cell_type_column, "not found in Seurat object metadata"))
  }
  message("✓ Cell type column found in metadata")
  
  message("=== EXTRACTING DATA ===")
  
  # Extract pseudotime for specified lineage
  pseudotime_matrix <- slingPseudotime(curves_obj)
  pseudotime_data <- pseudotime_matrix[, lineage_name]
  
  message(paste("Extracted pseudotime for", lineage_name, "- length:", length(pseudotime_data)))
  message(paste("Number of NA values:", sum(is.na(pseudotime_data))))
  message(paste("Number of finite values:", sum(is.finite(pseudotime_data))))
  if(sum(is.finite(pseudotime_data)) > 0) {
    message(paste("Pseudotime range:", paste(range(pseudotime_data, na.rm = TRUE), collapse = " to ")))
  }
  
  # Get gene expression
  gene_expression <- GetAssayData(seurat_obj, slot = expression_slot)[gene_name, ]
  message(paste("Extracted expression for", gene_name, "- length:", length(gene_expression)))
  
  # Get cell type information
  cell_types <- seurat_obj@meta.data[names(pseudotime_data), cell_type_column]
  message(paste("Extracted cell types - length:", length(cell_types)))
  message(paste("Number of NA cell types:", sum(is.na(cell_types))))
  message(paste("First few cell types:", paste(head(cell_types, na.rm = TRUE), collapse = ", ")))
  
  # Create data frame
  plot_data <- data.frame(
    pseudotime = pseudotime_data,
    expression = gene_expression[names(pseudotime_data)],
    cell_type = cell_types
  )
  
  message(paste("Created plot_data with", nrow(plot_data), "rows"))
  message(paste("Complete cases before filtering:", sum(complete.cases(plot_data))))
  
  # Remove NA values
  plot_data <- plot_data[complete.cases(plot_data), ]
  
  message(paste("Complete cases after filtering:", nrow(plot_data)))
  
  # Check if we have any valid data after removing NAs
  if (nrow(plot_data) == 0) {
    message("=== DEBUGGING INFO ===")
    message("Original pseudotime_data length:", length(pseudotime_data))
    message("Original gene_expression length:", length(gene_expression))
    message("Original cell_types length:", length(cell_types))
    message("Names match between pseudotime and expression:", 
            identical(names(pseudotime_data), names(gene_expression[names(pseudotime_data)])))
    message("Sample of pseudotime names:", paste(head(names(pseudotime_data)), collapse = ", "))
    message("Sample of expression names:", paste(head(names(gene_expression)), collapse = ", "))
    stop(paste("No valid data found for", lineage_name, ". Check if the lineage exists and has valid pseudotime values."))
  }
  
  # Check if pseudotime values are finite
  if (!all(is.finite(plot_data$pseudotime))) {
    stop("Non-finite pseudotime values found. Please check your slingshot curves object.")
  }
  
  message(paste("Found", nrow(plot_data), "cells with valid pseudotime and expression data for", lineage_name))
  
  # Count cells per cell type and identify small cell types
  cell_type_counts <- table(plot_data$cell_type)
  small_cell_types <- names(cell_type_counts[cell_type_counts < min_cells])
  
  message(paste("Cell type counts:"))
  print(cell_type_counts)
  message(paste("Small cell types (< ", min_cells, " cells):", paste(small_cell_types, collapse = ", ")))
  
  # Group small cell types
  plot_data$cell_type_plot <- as.character(plot_data$cell_type)
  plot_data$cell_type_plot[plot_data$cell_type %in% small_cell_types] <- paste0("Other (< ", min_cells, " cells)")
  
  # Order cell types by median pseudotime
  cell_type_order <- plot_data %>%
    group_by(cell_type_plot) %>%
    summarise(median_pseudotime = median(pseudotime, na.rm = TRUE), .groups = 'drop') %>%
    arrange(median_pseudotime) %>%
    pull(cell_type_plot)
  
  plot_data$cell_type_plot <- factor(plot_data$cell_type_plot, levels = cell_type_order)
  
  # Create pseudotime bins
  pseudotime_range <- range(plot_data$pseudotime, na.rm = TRUE)
  bin_breaks <- seq(pseudotime_range[1], pseudotime_range[2], length.out = n_bins + 1)
  plot_data$pseudotime_bin <- cut(plot_data$pseudotime, breaks = bin_breaks, include.lowest = TRUE)
  
  # Find dominant cell type in each bin (excluding small cell types)
  dominant_celltype <- plot_data %>%
    filter(cell_type_plot != paste0("Other (< ", min_cells, " cells)")) %>%
    group_by(pseudotime_bin) %>%
    count(cell_type_plot) %>%
    slice_max(n, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(pseudotime_bin, dominant_type = cell_type_plot)
  
  # Create bin centers
  bin_centers <- (bin_breaks[-1] + bin_breaks[-length(bin_breaks)]) / 2
  dominant_celltype$pseudotime_center <- bin_centers[as.numeric(dominant_celltype$pseudotime_bin)]
  
  # Fit GAM model for smooth predictions
  gam_model <- gam(expression ~ s(pseudotime, bs = "cs"), data = plot_data)
  smooth_data <- data.frame(pseudotime = bin_centers)
  smooth_data$expression <- predict(gam_model, newdata = smooth_data)
  
  # Merge with dominant cell types
  smooth_data <- merge(smooth_data, 
                       dominant_celltype[, c("pseudotime_center", "dominant_type")], 
                       by.x = "pseudotime", by.y = "pseudotime_center", all.x = TRUE)
  
  # Create segment groups to break lines
  smooth_data <- smooth_data[order(smooth_data$pseudotime), ]
  smooth_data$segment_group <- cumsum(c(TRUE, smooth_data$dominant_type[-1] != smooth_data$dominant_type[-nrow(smooth_data)] | 
                                       is.na(smooth_data$dominant_type[-1]) | is.na(smooth_data$dominant_type[-nrow(smooth_data)])))
  
  # Remove rows with NA dominant_type
  smooth_data <- smooth_data[!is.na(smooth_data$dominant_type), ]
  
  # Set up colors
  major_cell_types <- levels(plot_data$cell_type_plot)[levels(plot_data$cell_type_plot) != paste0("Other (< ", min_cells, " cells)")]
  
  if (is.null(color_palette)) {
    colors <- rainbow(length(major_cell_types))
  } else {
    if (length(color_palette) < length(major_cell_types)) {
      warning("Not enough colors in palette. Recycling colors.")
      colors <- rep(color_palette, length.out = length(major_cell_types))
    } else {
      colors <- color_palette[1:length(major_cell_types)]
    }
  }
  names(colors) <- major_cell_types
  
  # Create plot title with optional destination
  plot_title <- paste(gene_name, "Expression Trend Along", lineage_name)
  if (!is.null(lineage_destination)) {
    plot_title <- paste(gene_name, "Expression Trend Along", lineage_name, "-", lineage_destination)
  }
  
  # Create plot
  p <- ggplot() +
    # Add confidence interval
    geom_smooth(data = plot_data, aes(x = pseudotime, y = expression),
                method = "gam", formula = y ~ s(x, bs = "cs"), 
                se = TRUE, color = NA, fill = "darkgrey", alpha = 0.5) +
    # Add colored line segments
    geom_path(data = smooth_data, aes(x = pseudotime, y = expression, 
                                      color = dominant_type, group = segment_group), 
              linewidth = 2) +
    scale_color_manual(values = colors, name = "Dominant Cell Type") +
    labs(
      title = plot_title,
      x = "Pseudotime",
      y = paste(gene_name, "Expression"),
      subtitle = paste0("Line colored by dominant cell type in each pseudotime region (n = ", 
                       nrow(plot_data), " cells, min_cells = ", min_cells, ", bins = ", n_bins, ")")
    ) +
    theme_classic() +
    theme(
      plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 11, hjust = 0.5),
      axis.title = element_text(size = 12),
      axis.text = element_text(size = 10),
      legend.title = element_text(size = 11),
      legend.text = element_text(size = 9),
      panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
    ) +
    guides(color = guide_legend(override.aes = list(linewidth = 3)))
  
  # Add custom y-axis limits if provided
  if (!is.null(ylim)) {
    if (length(ylim) != 2 || !is.numeric(ylim)) {
      warning("ylim should be a numeric vector of length 2. Using automatic limits.")
    } else {
      p <- p + coord_cartesian(ylim = ylim)
    }
  }
  
  # Save plot if requested
  if (save_plot) {
    if (is.null(filename)) {
      filename <- paste0(gene_name, "_", lineage_name, "_pseudotime_trend.pdf")
    }
    ggsave(filename, plot = p, width = plot_width, height = plot_height)
    message(paste("Plot saved as:", filename))
  }
  
  return(p)
}

#' # With y-axis limits
#' p <- plot_gene_pseudotime(seurat_obj, curves_df_old_umap, "Atf3", "Lineage1",
#'                          ylim = c(0, 5))
#' 
#' # With lineage destination in title
#' p <- plot_gene_pseudotime(seurat_obj, curves_df_old_umap, "Atf3", "Lineage7",
#'                          lineage_destination = "Erythroid")
#' 
#' # With both options
#' p <- plot_gene_pseudotime(seurat_obj, curves_df_old_umap, "Sox2", "Lineage2",
#'                          ylim = c(0, 3), lineage_destination = "Neural")

In [ ]:
p_Mesenchyme   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Mesenchyme)

In [ ]:
p_Allantois   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage2",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
p_Allantois

In [ ]:
p_Epicardium   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage3",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
p_Epicardium

In [ ]:
p_limb   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage4",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
p_limb

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
saveRDS(xobj, "./atlas_latemesoderm_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_late_Mesoderm_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_late_mesoderm_slingshot_curves.rds')

In [ ]:
atlas_late_mesoderm <- xobj
atlas_late_mesoderm_lineages <- lineages_xobj
atlas_late_mesoderm_curves <- curves_xobj

In [ ]:
# after crush
atlas_lm_xobj <- readRDS('./atlas_latemesoderm_lineage.rds') # it should be v4 actually.
lineages_atlas_lm <- readRDS('./atlas_late_Mesoderm_slingshot_lin.rds')
curves_atlas_lm <- readRDS('./atlas_late_mesoderm_slingshot_curves.rds')

In [ ]:
atlas_lm_xobj

In [ ]:
lineages_atlas_lm

In [ ]:
curves_atlas_lm

### Cardiac Lineages

I tried with doing first and second heart fields together, but it didn't worked very well,now I am  doing them seperately.

#### FHF

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm','Lateral plate mesoderm','Cardiopharyngeal progenitors FHF','Cardiopharyngeal progenitors',
                                                              'Pharyngeal mesoderm','Anterior cardiopharyngeal progenitors','Cardiomyocytes FHF 1','Cardiomyocytes FHF 2' ))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas',label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('Cardiomyocytes FHF 2'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
p_cardiac_fhf   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_cardiac_fhf)

In [ ]:
saveRDS(xobj, "./atlas_cardiac_fhf_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_cardiac_fhf_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_cardiac_fhf_slingshot_curves.rds')

In [ ]:
# after crush
atlas_fhf_xobj <- readRDS('./atlas_cardiac_fhf_lineage.rds') # it should be v4 actually.
lineages_atlas_fhf <- readRDS('./atlas_cardiac_fhf_slingshot_lin.rds')
curves_atlas_fhf <- readRDS('./atlas_cardiac_fhf_slingshot_curves.rds')

In [ ]:
atlas_fhf_xobj

In [ ]:
lineages_atlas_fhf

In [ ]:
curves_atlas_fhf

#### SHF

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm','Lateral plate mesoderm','Cardiopharyngeal progenitors SHF','Cardiopharyngeal progenitors',
                                                              'Pharyngeal mesoderm','Anterior cardiopharyngeal progenitors','Cardiomyocytes SHF 1','Cardiomyocytes SHF 2' ))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas',label = TRUE)

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('Cardiomyocytes SHF 2'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
p_cardiac_shf   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_cardiac_shf)

In [ ]:
saveRDS(xobj, "./atlas_cardiac_shf_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_cardiac_shf_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_cardiac_shf_slingshot_curves.rds')

In [ ]:
# after crush
atlas_shf_xobj <- readRDS('./atlas_cardiac_shf_lineage.rds') 
lineages_atlas_shf <- readRDS('./atlas_cardiac_shf_slingshot_lin.rds')
curves_atlas_shf <- readRDS('./atlas_cardiac_shf_slingshot_curves.rds')

In [ ]:
atlas_shf_xobj

In [ ]:
lineages_atlas_shf

In [ ]:
curves_atlas_shf

## Blood

I tried with first and second wave together and it didn't worked. so i will do it with subsetting.

### Erythroid

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm',
                                                              'Haematoendothelial progenitors', 'Blood progenitors', 'Erythroid'                                                        
                                                          ))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas', label = TRUE)+NoLegend()

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('Erythroid'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
p_Erythroid   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Erythroid)

In [ ]:
getwd()

In [ ]:
saveRDS(xobj, "./atlas_erythroid_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_erythroid_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_erythroid_slingshot_curves.rds')

In [ ]:
atlas_erythroid_xobj <- xobj
lineages_atlas_erythroid <- lineages_xobj
curves_atlas_erythroid <- curves_xobj

In [ ]:
atlas_erythroid_xobj

In [ ]:
lineages_atlas_erythroid

In [ ]:
curves_atlas_erythroid

### Endothelium

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm',
                                                              'Haematoendothelial progenitors', 'Venous endothelium','YS endothelium' ,'Embryo proper endothelium'                                                       
                                                          ))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas', label = TRUE)+NoLegend()

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "slingshot", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('YS endothelium'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
p_endothelium   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_endothelium)

In [ ]:
getwd()

In [ ]:
saveRDS(xobj, "./atlas_endothelium_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_endothelium_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_endothelium_slingshot_curves.rds')

In [ ]:
atlas_endothelium_xobj <- xobj
lineages_atlas_endothelium <- lineages_xobj
curves_atlas_endothelium <- curves_xobj

In [ ]:
atlas_endothelium_xobj

In [ ]:
lineages_atlas_endothelium

In [ ]:
atlas_endothelium_xobj

### EMP

In [ ]:
atlas

In [ ]:
table(atlas$celltype_extended_atlas)

In [ ]:
xobj <- subset(atlas, subset = celltype_extended_atlas %in% c('Epiblast','Primitive Streak','Nascent mesoderm',
                                                              'Haematoendothelial progenitors', 'Venous endothelium','YS endothelium' ,'Embryo proper endothelium','EMP'
                                                             ))

In [ ]:
xobj

In [ ]:
table(xobj$celltype_extended_atlas)

In [ ]:
Idents(xobj) <- xobj$celltype_extended_atlas

In [ ]:
xobj <- subset(xobj, downsample = 2000)
xobj

In [ ]:
DimPlot(xobj, reduction = "UMAP")

In [ ]:
# New harmony Umap
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L,
                     n.neighbors = 10,reduction.name = "xobj_umap0_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.5, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap1_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.4, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap2_int")
xobj <- RunUMAP(xobj, reduction = "PCA", dims = 1:75, n.components = 2L, local.connectivity = 1L,
                  min.dist = 0.3, spread = 0.5,
                     n.neighbors = 10,reduction.name = "xobj_umap3_int")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
DimPlot(xobj, reduction = "UMAP", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap0_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap2_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')
DimPlot(xobj, reduction = "xobj_umap3_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas')

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
DimPlot(xobj, reduction = "xobj_umap1_int", cols = r3dcol$cols_94, group.by = 'celltype_extended_atlas', label = TRUE)+NoLegend()

In [ ]:
set.seed(1)
lineages_xobj <- as.SlingshotDataSet(getLineages(
  data           = xobj@reductions$xobj_umap1_int@cell.embeddings,
  clusterLabels  = xobj$celltype_extended_atlas,
  dist.method    = "mnn", # It can be: "simple", "scaled.full", "scaled.diag", "slingshot" or "mnn"
  end.clus       = c('EMP'),
  start.clus     = "Epiblast"# define where to START the trajectories
)) 

In [ ]:
# Define curves
curves_xobj <- as.SlingshotDataSet(getCurves(data = lineages_xobj,approx_points = 1000, thresh = 1))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p_curves <-plot_umap_with_curves(seurat_obj = xobj,
  reduction = "xobj_umap1_int",
  cluster_col = "celltype_extended_atlas",
  color_vector = rep(r3dcol$cols_46,2),
  curves = curves_xobj,        # or 'curves' object from Slingshot
  label_cex = 1,
  point_pch = 16,
  point_cex = 1,
  line_lwd = 2
)
p_curves

In [ ]:
curves_xobj

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
p_pseudo <- plot_umap_pseudotime_and_centroids(
  seurat_obj = xobj,
  curves = curves_xobj,
  embedding = "xobj_umap1_int",
 cluster_col = "celltype_extended_atlas",
  color_vector = r3dcol$cols_94,   # Or your own color palette
  point_cex = 0.4,
  label_cex = 1,
  legend_title = "Pseudotime",
  legend_mar = 2
)
p_pseudo

In [ ]:
p_emp   <- plot_gene_pseudotime(
   seurat_obj = xobj,
   curves_obj = curves_xobj,
   gene_name = "Atf3",
   lineage_name = "Lineage1",
   min_cells = 100,
   n_bins = 200,
   color_palette = r3dcol$cols_94,
   cell_type_column = "celltype_extended_atlas"
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_emp)

In [ ]:
getwd()

In [ ]:
saveRDS(xobj, "./atlas_emp_lineage.rds")

In [ ]:
saveRDS(lineages_xobj, './atlas_emp_slingshot_lin.rds')
saveRDS(curves_xobj, './atlas_emp_slingshot_curves.rds')

In [ ]:
atlas_emp_xobj <- xobj
lineages_atlas_emp <- lineages_xobj
curves_atlas_emp <- curves_xobj

In [ ]:
atlas_endothelium_xobj

In [ ]:
lineages_atlas_endothelium

In [ ]:
atlas_endothelium_xobj

# plotting code

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)
library(dplyr)

#' Utility function to extract lineage endpoint from curves object
#'
#' @param curves_obj Slingshot curves object
#' @param lineage_name Character. Name of the lineage
#' @param seurat_obj Seurat object for cell type information
#' @param cell_type_column Character. Column name for cell types
#'
get_lineage_endpoint <- function(curves_obj, lineage_name, seurat_obj, cell_type_column) {
  tryCatch({
    # Method 1: Try to get endpoint from slingshot lineages
    if (exists("slingLineages", where = "package:slingshot")) {
      lineages <- slingLineages(curves_obj)
      if (lineage_name %in% names(lineages)) {
        lineage_clusters <- lineages[[lineage_name]]
        endpoint_cluster <- lineage_clusters[length(lineage_clusters)]
        return(paste("Cluster", endpoint_cluster))
      }
    }
    
    # Method 2: Find endpoint based on maximum pseudotime
    pseudotime_data <- slingPseudotime(curves_obj)[, lineage_name]
    
    # Get cells at the end of pseudotime (top 5% of pseudotime values)
    max_pseudotime <- max(pseudotime_data, na.rm = TRUE)
    high_pseudotime_threshold <- max_pseudotime * 0.95
    
    endpoint_cells <- names(pseudotime_data)[pseudotime_data >= high_pseudotime_threshold & !is.na(pseudotime_data)]
    
    if (length(endpoint_cells) > 0) {
      # Get cell types of endpoint cells
      endpoint_cell_types <- seurat_obj@meta.data[endpoint_cells, cell_type_column]
      
      # Find most common cell type at endpoint
      cell_type_counts <- table(endpoint_cell_types)
      most_common_endpoint <- names(cell_type_counts)[which.max(cell_type_counts)]
      
      return(most_common_endpoint)
    }
    
    return(NULL)
    
  }, error = function(e) {
    message("Could not automatically determine lineage endpoint: ", e$message)
    return(NULL)
  })
}
#'
#' @param seurat_obj Seurat object
#' @param curves_obj Slingshot curves object
#' @param cell_type_column Character. Column name for cell types
#'
check_pseudotime_data <- function(seurat_obj, curves_obj, cell_type_column = "seurat_clusters") {
  
  cat("=== PSEUDOTIME DATA DIAGNOSTIC ===\n")
  
  # Check available lineages
  lineages <- colnames(slingPseudotime(curves_obj))
  cat("Available lineages:", paste(lineages, collapse = ", "), "\n")
  
  # Check pseudotime data for each lineage
  pseudotime_matrix <- slingPseudotime(curves_obj, na = FALSE)
  
  for (lineage in lineages) {
    pseudotime_values <- pseudotime_matrix[, lineage]
    n_total <- length(pseudotime_values)
    n_finite <- sum(is.finite(pseudotime_values))
    n_na <- sum(is.na(pseudotime_values))
    
    cat("\n", lineage, ":\n")
    cat("  Total cells:", n_total, "\n")
    cat("  Finite values:", n_finite, "\n")
    cat("  NA values:", n_na, "\n")
    
    if (n_finite > 0) {
      cat("  Range:", round(range(pseudotime_values, na.rm = TRUE), 3), "\n")
    }
  }
  
  # Check cell name overlap
  curve_cells <- rownames(pseudotime_matrix)
  seurat_cells <- colnames(seurat_obj)
  overlap <- intersect(curve_cells, seurat_cells)
  
  cat("\n=== CELL NAME OVERLAP ===\n")
  cat("Cells in curves object:", length(curve_cells), "\n")
  cat("Cells in Seurat object:", length(seurat_cells), "\n")
  cat("Overlapping cells:", length(overlap), "\n")
  
  if (length(overlap) == 0) {
    cat("WARNING: No overlapping cell names found!\n")
    cat("First 5 curve cell names:", head(curve_cells, 5), "\n")
    cat("First 5 Seurat cell names:", head(seurat_cells, 5), "\n")
  }
  
  return(invisible(NULL))
}

#' Plot Gene Expression Along Pseudotime with Cell Type Coloring
#'
#' This function creates a trend plot showing gene expression along a specific lineage
#' pseudotime trajectory, with the trend line colored by dominant cell types in each region.
#'
#' @param seurat_obj Seurat object containing gene expression data
#' @param curves_obj Slingshot curves object containing pseudotime information
#' @param gene_name Character. Name of the gene to plot (must match gene names in Seurat object)
#' @param lineage_name Character. Name of the lineage to plot (e.g., "Lineage1", "Lineage2")
#' @param min_cells Numeric. Minimum number of cells required for a cell type to be considered major (default: 50)
#' @param n_bins Numeric. Number of bins to divide pseudotime axis for determining dominant cell types (default: 50)
#' @param color_palette Character vector. Colors to use for major cell types. If NULL, uses rainbow colors (default: NULL)
#' @param cell_type_column Character. Column name in Seurat object metadata containing cell type information (default: "seurat_clusters")
#' @param expression_slot Character. Which slot to use for gene expression ("data", "counts", or "scale.data") (default: "data")
#' @param ylim Numeric vector of length 2. Y-axis limits c(min, max). If NULL, uses automatic limits (default: NULL)
#' @param lineage_destination Character. Optional destination/endpoint name for the lineage (e.g., "Erythroid", "Neural"). If NULL, function will attempt to automatically determine from curves object. Set to FALSE to disable endpoint detection (default: NULL)
#' @param save_plot Logical. Whether to save the plot (default: FALSE)
#' @param filename Character. Filename for saving plot (default: NULL)
#' @param plot_width Numeric. Width for saved plot (default: 10)
#' @param plot_height Numeric. Height for saved plot (default: 6)
#'
#' @return ggplot object
#'
plot_gene_pseudotime <- function(seurat_obj,
                                curves_obj,
                                gene_name,
                                lineage_name,
                                min_cells = 50,
                                n_bins = 50,
                                color_palette = NULL,
                                cell_type_column = "seurat_clusters",
                                expression_slot = "data",
                                ylim = NULL,
                                lineage_destination = NULL,
                                save_plot = FALSE,
                                filename = NULL,
                                plot_width = 10,
                                plot_height = 6) {
  
  # Input validation
  message("=== STARTING INPUT VALIDATION ===")
  
  if (!gene_name %in% rownames(seurat_obj)) {
    stop(paste("Gene", gene_name, "not found in Seurat object"))
  }
  message("✓ Gene found in Seurat object")
  
  if (!lineage_name %in% colnames(slingPseudotime(curves_obj))) {
    available_lineages <- colnames(slingPseudotime(curves_obj))
    stop(paste("Lineage", lineage_name, "not found. Available lineages:", paste(available_lineages, collapse = ", ")))
  }
  message("✓ Lineage found in curves object")
  
  if (!cell_type_column %in% colnames(seurat_obj@meta.data)) {
    stop(paste("Cell type column", cell_type_column, "not found in Seurat object metadata"))
  }
  message("✓ Cell type column found in metadata")
  
  message("=== EXTRACTING DATA ===")
  
  # Extract pseudotime for specified lineage
  pseudotime_matrix <- slingPseudotime(curves_obj)
  pseudotime_data <- pseudotime_matrix[, lineage_name]
  
  message(paste("Extracted pseudotime for", lineage_name, "- length:", length(pseudotime_data)))
  message(paste("Number of NA values:", sum(is.na(pseudotime_data))))
  message(paste("Number of finite values:", sum(is.finite(pseudotime_data))))
  if(sum(is.finite(pseudotime_data)) > 0) {
    message(paste("Pseudotime range:", paste(range(pseudotime_data, na.rm = TRUE), collapse = " to ")))
  }
  
  # Get gene expression
  gene_expression <- GetAssayData(seurat_obj, slot = expression_slot)[gene_name, ]
  message(paste("Extracted expression for", gene_name, "- length:", length(gene_expression)))
  
  # Get cell type information
  cell_types <- seurat_obj@meta.data[names(pseudotime_data), cell_type_column]
  message(paste("Extracted cell types - length:", length(cell_types)))
  message(paste("Number of NA cell types:", sum(is.na(cell_types))))
  message(paste("First few cell types:", paste(head(cell_types, na.rm = TRUE), collapse = ", ")))
  
  # Create data frame
  plot_data <- data.frame(
    pseudotime = pseudotime_data,
    expression = gene_expression[names(pseudotime_data)],
    cell_type = cell_types
  )
  
  message(paste("Created plot_data with", nrow(plot_data), "rows"))
  message(paste("Complete cases before filtering:", sum(complete.cases(plot_data))))
  
  # Remove NA values
  plot_data <- plot_data[complete.cases(plot_data), ]
  
  message(paste("Complete cases after filtering:", nrow(plot_data)))
  
  # Check if we have any valid data after removing NAs
  if (nrow(plot_data) == 0) {
    message("=== DEBUGGING INFO ===")
    message("Original pseudotime_data length:", length(pseudotime_data))
    message("Original gene_expression length:", length(gene_expression))
    message("Original cell_types length:", length(cell_types))
    message("Names match between pseudotime and expression:", 
            identical(names(pseudotime_data), names(gene_expression[names(pseudotime_data)])))
    message("Sample of pseudotime names:", paste(head(names(pseudotime_data)), collapse = ", "))
    message("Sample of expression names:", paste(head(names(gene_expression)), collapse = ", "))
    stop(paste("No valid data found for", lineage_name, ". Check if the lineage exists and has valid pseudotime values."))
  }
  
  # Check if pseudotime values are finite
  if (!all(is.finite(plot_data$pseudotime))) {
    stop("Non-finite pseudotime values found. Please check your slingshot curves object.")
  }
  
  message(paste("Found", nrow(plot_data), "cells with valid pseudotime and expression data for", lineage_name))
  
  # Count cells per cell type and identify small cell types
  cell_type_counts <- table(plot_data$cell_type)
  small_cell_types <- names(cell_type_counts[cell_type_counts < min_cells])
  
  message(paste("Cell type counts:"))
  print(cell_type_counts)
  message(paste("Small cell types (< ", min_cells, " cells):", paste(small_cell_types, collapse = ", ")))
  
  # Group small cell types
  plot_data$cell_type_plot <- as.character(plot_data$cell_type)
  plot_data$cell_type_plot[plot_data$cell_type %in% small_cell_types] <- paste0("Other (< ", min_cells, " cells)")
  
  # Order cell types by median pseudotime
  cell_type_order <- plot_data %>%
    group_by(cell_type_plot) %>%
    summarise(median_pseudotime = median(pseudotime, na.rm = TRUE), .groups = 'drop') %>%
    arrange(median_pseudotime) %>%
    pull(cell_type_plot)
  
  plot_data$cell_type_plot <- factor(plot_data$cell_type_plot, levels = cell_type_order)
  
  # Create pseudotime bins
  pseudotime_range <- range(plot_data$pseudotime, na.rm = TRUE)
  bin_breaks <- seq(pseudotime_range[1], pseudotime_range[2], length.out = n_bins + 1)
  plot_data$pseudotime_bin <- cut(plot_data$pseudotime, breaks = bin_breaks, include.lowest = TRUE)
  
  # Find dominant cell type in each bin (excluding small cell types)
  dominant_celltype <- plot_data %>%
    filter(cell_type_plot != paste0("Other (< ", min_cells, " cells)")) %>%
    group_by(pseudotime_bin) %>%
    count(cell_type_plot) %>%
    slice_max(n, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(pseudotime_bin, dominant_type = cell_type_plot)
  
  # Create bin centers
  bin_centers <- (bin_breaks[-1] + bin_breaks[-length(bin_breaks)]) / 2
  dominant_celltype$pseudotime_center <- bin_centers[as.numeric(dominant_celltype$pseudotime_bin)]
  
  # Fit GAM model for smooth predictions
  gam_model <- gam(expression ~ s(pseudotime, bs = "cs"), data = plot_data)
  smooth_data <- data.frame(pseudotime = bin_centers)
  smooth_data$expression <- predict(gam_model, newdata = smooth_data)
  
  # Merge with dominant cell types
  smooth_data <- merge(smooth_data, 
                       dominant_celltype[, c("pseudotime_center", "dominant_type")], 
                       by.x = "pseudotime", by.y = "pseudotime_center", all.x = TRUE)
  
  # Create segment groups to break lines
  smooth_data <- smooth_data[order(smooth_data$pseudotime), ]
  smooth_data$segment_group <- cumsum(c(TRUE, smooth_data$dominant_type[-1] != smooth_data$dominant_type[-nrow(smooth_data)] | 
                                       is.na(smooth_data$dominant_type[-1]) | is.na(smooth_data$dominant_type[-nrow(smooth_data)])))
  
  # Remove rows with NA dominant_type
  smooth_data <- smooth_data[!is.na(smooth_data$dominant_type), ]
  
  # Set up colors
  major_cell_types <- levels(plot_data$cell_type_plot)[levels(plot_data$cell_type_plot) != paste0("Other (< ", min_cells, " cells)")]
  
  if (is.null(color_palette)) {
    colors <- rainbow(length(major_cell_types))
  } else {
    if (length(color_palette) < length(major_cell_types)) {
      warning("Not enough colors in palette. Recycling colors.")
      colors <- rep(color_palette, length.out = length(major_cell_types))
    } else {
      colors <- color_palette[1:length(major_cell_types)]
    }
  }
  names(colors) <- major_cell_types
  
  message("=== DETERMINING LINEAGE ENDPOINT ===")
  
  # Determine lineage endpoint
  endpoint_name <- NULL
  if (is.null(lineage_destination)) {
    # Try to automatically determine endpoint
    endpoint_name <- get_lineage_endpoint(curves_obj, lineage_name, seurat_obj, cell_type_column)
    if (!is.null(endpoint_name)) {
      message(paste("✓ Automatically detected endpoint:", endpoint_name))
    } else {
      message("✗ Could not automatically determine endpoint")
    }
  } else if (lineage_destination != FALSE) {
    # Use provided endpoint
    endpoint_name <- lineage_destination
    message(paste("✓ Using provided endpoint:", endpoint_name))
  } else {
    # Explicitly disabled endpoint detection
    message("✓ Endpoint detection disabled")
  }
  
  # Create plot title with optional destination
  plot_title <- paste(gene_name, "Expression Trend Along", lineage_name)
  if (!is.null(endpoint_name)) {
    plot_title <- paste(gene_name, "Expression Trend Along", lineage_name, "-", endpoint_name)
  }
  
  # Create plot
  p <- ggplot() +
    # Add confidence interval
    geom_smooth(data = plot_data, aes(x = pseudotime, y = expression),
                method = "gam", formula = y ~ s(x, bs = "cs"), 
                se = TRUE, color = NA, fill = "darkgrey", alpha = 0.5) +
    # Add colored line segments
    geom_path(data = smooth_data, aes(x = pseudotime, y = expression, 
                                      color = dominant_type, group = segment_group), 
              linewidth = 2) +
    scale_color_manual(values = colors, name = "Dominant Cell Type") +
    labs(
      title = plot_title,
      x = "Pseudotime",
      y = paste(gene_name, "Expression"),
      subtitle = paste0("Line colored by dominant cell type in each pseudotime region (n = ", 
                       nrow(plot_data), " cells, min_cells = ", min_cells, ", bins = ", n_bins, ")")
    ) +
    theme_classic() +
    theme(
      plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 11, hjust = 0.5),
      axis.title = element_text(size = 12),
      axis.text = element_text(size = 10),
      legend.title = element_text(size = 11),
      legend.text = element_text(size = 9),
      panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
    ) +
    guides(color = guide_legend(override.aes = list(linewidth = 3)))
  
  # Add custom y-axis limits if provided
  if (!is.null(ylim)) {
    if (length(ylim) != 2 || !is.numeric(ylim)) {
      warning("ylim should be a numeric vector of length 2. Using automatic limits.")
    } else {
      p <- p + coord_cartesian(ylim = ylim)
    }
  }
  
  # Save plot if requested
  if (save_plot) {
    if (is.null(filename)) {
      filename <- paste0(gene_name, "_", lineage_name, "_pseudotime_trend.pdf")
    }
    ggsave(filename, plot = p, width = plot_width, height = plot_height)
    message(paste("Plot saved as:", filename))
  }
  
  return(p)
}

#' # Complete usage example with all parameters
#' p <- plot_gene_pseudotime(
#'   seurat_obj = df,                                    # Your Seurat object
#'   curves_obj = curves_df_old_umap,                   # Your slingshot curves object
#'   gene_name = "Atf3",                                # Gene to plot
#'   lineage_name = "Lineage2",                         # Which lineage trajectory
#'   min_cells = 100,                                   # Minimum cells for major cell types
#'   n_bins = 30,                                       # Number of pseudotime bins
#'   color_palette = r3dcol$cols_94,                    # Custom color palette
#'   cell_type_column = "celltype_PijuanSala2019",      # Cell type annotation column
#'   expression_slot = "data",                          # Expression data slot
#'   ylim = c(0, 4),                                    # Y-axis limits
#'   lineage_destination = "Neural Crest",              # Override auto-detected endpoint
#'   save_plot = TRUE,                                  # Save the plot
#'   filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename
#'   plot_width = 12,                                   # Plot width in inches
#'   plot_height = 8                                    # Plot height in inches
#' )
#' 
#' # Minimal usage (uses all defaults)
#' p_minimal <- plot_gene_pseudotime(df, curves_df_old_umap, "Atf3", "Lineage1")

In [ ]:
xobj$seurat_clusters <- xobj$celltype_extended_atlas

In [ ]:
p <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = xobj,                                    # Your Seurat object
  curves_obj = curves_xobj,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = r3dcol$cols_94,                    # Custom color palette
 # ylim = c(0, 4),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p)

## v30

In [ ]:
library(ggplot2)
library(slingshot)
library(Seurat)
library(mgcv)
library(dplyr)

#' Utility function to extract lineage endpoint from curves object
#'
#' @param curves_obj Slingshot curves object
#' @param lineage_name Character. Name of the lineage
#' @param seurat_obj Seurat object for cell type information
#' @param cell_type_column Character. Column name for cell types
#'
get_lineage_endpoint <- function(curves_obj, lineage_name, seurat_obj, cell_type_column) {
  tryCatch({
    # Method 1: Try to get endpoint from slingshot lineages
    if (exists("slingLineages", where = "package:slingshot")) {
      lineages <- slingLineages(curves_obj)
      if (lineage_name %in% names(lineages)) {
        lineage_clusters <- lineages[[lineage_name]]
        endpoint_cluster <- lineage_clusters[length(lineage_clusters)]
        return(paste("Cluster", endpoint_cluster))
      }
    }
    
    # Method 2: Find endpoint based on maximum pseudotime
    pseudotime_data <- slingPseudotime(curves_obj)[, lineage_name]
    
    # Get cells at the end of pseudotime (top 5% of pseudotime values)
    max_pseudotime <- max(pseudotime_data, na.rm = TRUE)
    high_pseudotime_threshold <- max_pseudotime * 0.95
    
    endpoint_cells <- names(pseudotime_data)[pseudotime_data >= high_pseudotime_threshold & !is.na(pseudotime_data)]
    
    if (length(endpoint_cells) > 0) {
      # Get cell types of endpoint cells
      endpoint_cell_types <- seurat_obj@meta.data[endpoint_cells, cell_type_column]
      
      # Find most common cell type at endpoint
      cell_type_counts <- table(endpoint_cell_types)
      most_common_endpoint <- names(cell_type_counts)[which.max(cell_type_counts)]
      
      return(most_common_endpoint)
    }
    
    return(NULL)
    
  }, error = function(e) {
    message("Could not automatically determine lineage endpoint: ", e$message)
    return(NULL)
  })
}
#'
#' @param seurat_obj Seurat object
#' @param curves_obj Slingshot curves object
#' @param cell_type_column Character. Column name for cell types
#'
check_pseudotime_data <- function(seurat_obj, curves_obj, cell_type_column = "seurat_clusters") {
  
  cat("=== PSEUDOTIME DATA DIAGNOSTIC ===\n")
  
  # Check available lineages
  lineages <- colnames(slingPseudotime(curves_obj))
  cat("Available lineages:", paste(lineages, collapse = ", "), "\n")
  
  # Check pseudotime data for each lineage
  pseudotime_matrix <- slingPseudotime(curves_obj, na = FALSE)
  
  for (lineage in lineages) {
    pseudotime_values <- pseudotime_matrix[, lineage]
    n_total <- length(pseudotime_values)
    n_finite <- sum(is.finite(pseudotime_values))
    n_na <- sum(is.na(pseudotime_values))
    
    cat("\n", lineage, ":\n")
    cat("  Total cells:", n_total, "\n")
    cat("  Finite values:", n_finite, "\n")
    cat("  NA values:", n_na, "\n")
    
    if (n_finite > 0) {
      cat("  Range:", round(range(pseudotime_values, na.rm = TRUE), 3), "\n")
    }
  }
  
  # Check cell name overlap
  curve_cells <- rownames(pseudotime_matrix)
  seurat_cells <- colnames(seurat_obj)
  overlap <- intersect(curve_cells, seurat_cells)
  
  cat("\n=== CELL NAME OVERLAP ===\n")
  cat("Cells in curves object:", length(curve_cells), "\n")
  cat("Cells in Seurat object:", length(seurat_cells), "\n")
  cat("Overlapping cells:", length(overlap), "\n")
  
  if (length(overlap) == 0) {
    cat("WARNING: No overlapping cell names found!\n")
    cat("First 5 curve cell names:", head(curve_cells, 5), "\n")
    cat("First 5 Seurat cell names:", head(seurat_cells, 5), "\n")
  }
  
  return(invisible(NULL))
}

#' Plot Gene Expression Along Pseudotime with Cell Type Coloring
#'
#' This function creates a trend plot showing gene expression along a specific lineage
#' pseudotime trajectory, with the trend line colored by dominant cell types in each region.
#'
#' @param seurat_obj Seurat object containing gene expression data
#' @param curves_obj Slingshot curves object containing pseudotime information
#' @param gene_name Character. Name of the gene to plot (must match gene names in Seurat object)
#' @param lineage_name Character. Name of the lineage to plot (e.g., "Lineage1", "Lineage2")
#' @param min_cells Numeric. Minimum number of cells required for a cell type to be considered major (default: 50)
#' @param n_bins Numeric. Number of bins to divide pseudotime axis for determining dominant cell types (default: 50)
#' @param color_palette Character vector. Colors to use for major cell types. If NULL, uses rainbow colors (default: NULL)
#' @param cell_type_column Character. Column name in Seurat object metadata containing cell type information (default: "seurat_clusters")
#' @param expression_slot Character. Which slot to use for gene expression ("data", "counts", or "scale.data") (default: "data")
#' @param exclude_cell_types Character vector. Cell types to completely remove from analysis and plotting (default: NULL)
#' @param cell_type_order Character vector. Custom order for cell types. If NULL, uses pseudotime-based ordering (default: NULL)
#' @param ylim Numeric vector of length 2. Y-axis limits c(min, max). If NULL, uses automatic limits (default: NULL)
#' @param lineage_destination Character. Optional destination/endpoint name for the lineage (e.g., "Erythroid", "Neural"). If NULL, function will attempt to automatically determine from curves object. Set to FALSE to disable endpoint detection (default: NULL)
#' @param show_subtitle Logical. Whether to show the subtitle with analysis details (default: TRUE)
#' @param save_plot Logical. Whether to save the plot (default: FALSE)
#' @param filename Character. Filename for saving plot (default: NULL)
#' @param plot_width Numeric. Width for saved plot (default: 10)
#' @param plot_height Numeric. Height for saved plot (default: 6)
#'
#' @return ggplot object
#'
plot_gene_pseudotime <- function(seurat_obj,
                                curves_obj,
                                gene_name,
                                lineage_name,
                                min_cells = 50,
                                n_bins = 50,
                                color_palette = NULL,
                                cell_type_column = "seurat_clusters",
                                expression_slot = "data",
                                exclude_cell_types = NULL,
                                cell_type_order = NULL,
                                ylim = NULL,
                                lineage_destination = NULL,
                                show_subtitle = TRUE,
                                save_plot = FALSE,
                                filename = NULL,
                                plot_width = 10,
                                plot_height = 6) {
  
  # Input validation
  message("=== STARTING INPUT VALIDATION ===")
  
  if (!gene_name %in% rownames(seurat_obj)) {
    stop(paste("Gene", gene_name, "not found in Seurat object"))
  }
  message("✓ Gene found in Seurat object")
  
  if (!lineage_name %in% colnames(slingPseudotime(curves_obj))) {
    available_lineages <- colnames(slingPseudotime(curves_obj))
    stop(paste("Lineage", lineage_name, "not found. Available lineages:", paste(available_lineages, collapse = ", ")))
  }
  message("✓ Lineage found in curves object")
  
  if (!cell_type_column %in% colnames(seurat_obj@meta.data)) {
    stop(paste("Cell type column", cell_type_column, "not found in Seurat object metadata"))
  }
  message("✓ Cell type column found in metadata")
  
  message("=== EXTRACTING DATA ===")
  
  # Extract pseudotime for specified lineage
  pseudotime_matrix <- slingPseudotime(curves_obj)
  pseudotime_data <- pseudotime_matrix[, lineage_name]
  
  message(paste("Extracted pseudotime for", lineage_name, "- length:", length(pseudotime_data)))
  message(paste("Number of NA values:", sum(is.na(pseudotime_data))))
  message(paste("Number of finite values:", sum(is.finite(pseudotime_data))))
  if(sum(is.finite(pseudotime_data)) > 0) {
    message(paste("Pseudotime range:", paste(range(pseudotime_data, na.rm = TRUE), collapse = " to ")))
  }
  
  # Get gene expression
  gene_expression <- GetAssayData(seurat_obj, slot = expression_slot)[gene_name, ]
  message(paste("Extracted expression for", gene_name, "- length:", length(gene_expression)))
  
  # Get cell type information
  cell_types <- seurat_obj@meta.data[names(pseudotime_data), cell_type_column]
  message(paste("Extracted cell types - length:", length(cell_types)))
  message(paste("Number of NA cell types:", sum(is.na(cell_types))))
  message(paste("First few cell types:", paste(head(cell_types, na.rm = TRUE), collapse = ", ")))
  
  # Create data frame
  plot_data <- data.frame(
    pseudotime = pseudotime_data,
    expression = gene_expression[names(pseudotime_data)],
    cell_type = cell_types
  )
  
  message(paste("Created plot_data with", nrow(plot_data), "rows"))
  message(paste("Complete cases before filtering:", sum(complete.cases(plot_data))))
  
  # Remove NA values
  plot_data <- plot_data[complete.cases(plot_data), ]
  
  message(paste("Complete cases after filtering:", nrow(plot_data)))
  
  # Check if we have any valid data after removing NAs
  if (nrow(plot_data) == 0) {
    message("=== DEBUGGING INFO ===")
    message("Original pseudotime_data length:", length(pseudotime_data))
    message("Original gene_expression length:", length(gene_expression))
    message("Original cell_types length:", length(cell_types))
    message("Names match between pseudotime and expression:", 
            identical(names(pseudotime_data), names(gene_expression[names(pseudotime_data)])))
    message("Sample of pseudotime names:", paste(head(names(pseudotime_data)), collapse = ", "))
    message("Sample of expression names:", paste(head(names(gene_expression)), collapse = ", "))
    stop(paste("No valid data found for", lineage_name, ". Check if the lineage exists and has valid pseudotime values."))
  }
  
  # Check if pseudotime values are finite
  if (!all(is.finite(plot_data$pseudotime))) {
    stop("Non-finite pseudotime values found. Please check your slingshot curves object.")
  }
  
  message(paste("Found", nrow(plot_data), "cells with valid pseudotime and expression data for", lineage_name))
  
  # Remove excluded cell types completely
  if (!is.null(exclude_cell_types)) {
    n_before <- nrow(plot_data)
    plot_data <- plot_data[!plot_data$cell_type %in% exclude_cell_types, ]
    n_after <- nrow(plot_data)
    n_removed <- n_before - n_after
    
    message(paste("Excluded cell types:", paste(exclude_cell_types, collapse = ", ")))
    message(paste("Removed", n_removed, "cells belonging to excluded cell types"))
    message(paste("Remaining cells for analysis:", n_after))
    
    # Check if we still have data after exclusions
    if (nrow(plot_data) == 0) {
      stop("No cells remaining after excluding specified cell types.")
    }
  }
  
  # Count cells per cell type and identify small cell types (after exclusions)
  cell_type_counts <- table(plot_data$cell_type)
  small_cell_types <- names(cell_type_counts[cell_type_counts < min_cells])
  
  message(paste("Cell type counts after exclusions:"))
  print(cell_type_counts)
  message(paste("Small cell types (< ", min_cells, " cells):", paste(small_cell_types, collapse = ", ")))
  
  # Group only small cell types (excluded types are already removed)
  plot_data$cell_type_plot <- as.character(plot_data$cell_type)
  plot_data$cell_type_plot[plot_data$cell_type %in% small_cell_types] <- paste0("Other (< ", min_cells, " cells)")
  
  # Get major cell types after exclusions
  major_cell_types_raw <- setdiff(unique(plot_data$cell_type_plot), paste0("Other (< ", min_cells, " cells)"))
  
  # Order cell types
  if (!is.null(cell_type_order)) {
    # Use custom ordering
    # Check if all specified cell types exist (after exclusions)
    available_major_types <- intersect(cell_type_order, major_cell_types_raw)
    missing_types <- setdiff(cell_type_order, major_cell_types_raw)
    extra_types <- setdiff(major_cell_types_raw, cell_type_order)
    
    if (length(missing_types) > 0) {
      message(paste("Warning: Some cell types in custom order not found after exclusions:", paste(missing_types, collapse = ", ")))
    }
    if (length(extra_types) > 0) {
      message(paste("Note: Adding unspecified major cell types at end:", paste(extra_types, collapse = ", ")))
    }
    
    # Create final ordering
    cell_type_order_final <- c(available_major_types, extra_types)
    
    # Add "Other" category if it exists
    if (paste0("Other (< ", min_cells, " cells)") %in% plot_data$cell_type_plot) {
      cell_type_order_final <- c(cell_type_order_final, paste0("Other (< ", min_cells, " cells)"))
    }
    
    message(paste("Using custom cell type order:", paste(cell_type_order_final, collapse = " -> ")))
    
  } else {
    # Use pseudotime-based ordering (default behavior)
    cell_type_order_final <- plot_data %>%
      group_by(cell_type_plot) %>%
      summarise(median_pseudotime = median(pseudotime, na.rm = TRUE), .groups = 'drop') %>%
      arrange(median_pseudotime) %>%
      pull(cell_type_plot)
    
    message(paste("Using pseudotime-based cell type order:", paste(cell_type_order_final, collapse = " -> ")))
  }
  
  plot_data$cell_type_plot <- factor(plot_data$cell_type_plot, levels = cell_type_order_final)
  
  # Create pseudotime bins
  pseudotime_range <- range(plot_data$pseudotime, na.rm = TRUE)
  bin_breaks <- seq(pseudotime_range[1], pseudotime_range[2], length.out = n_bins + 1)
  plot_data$pseudotime_bin <- cut(plot_data$pseudotime, breaks = bin_breaks, include.lowest = TRUE)
  
  # Find dominant cell type in each bin (excluding small cell types)
  dominant_celltype <- plot_data %>%
    filter(cell_type_plot != paste0("Other (< ", min_cells, " cells)")) %>%
    group_by(pseudotime_bin) %>%
    count(cell_type_plot) %>%
    slice_max(n, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(pseudotime_bin, dominant_type = cell_type_plot)
  
  # Create bin centers
  bin_centers <- (bin_breaks[-1] + bin_breaks[-length(bin_breaks)]) / 2
  dominant_celltype$pseudotime_center <- bin_centers[as.numeric(dominant_celltype$pseudotime_bin)]
  
  # Fit GAM model for smooth predictions
  gam_model <- gam(expression ~ s(pseudotime, bs = "cs"), data = plot_data)
  smooth_data <- data.frame(pseudotime = bin_centers)
  smooth_data$expression <- predict(gam_model, newdata = smooth_data)
  
  # Merge with dominant cell types
  smooth_data <- merge(smooth_data, 
                       dominant_celltype[, c("pseudotime_center", "dominant_type")], 
                       by.x = "pseudotime", by.y = "pseudotime_center", all.x = TRUE)
  
  # Create segment groups to break lines
  smooth_data <- smooth_data[order(smooth_data$pseudotime), ]
  smooth_data$segment_group <- cumsum(c(TRUE, smooth_data$dominant_type[-1] != smooth_data$dominant_type[-nrow(smooth_data)] | 
                                       is.na(smooth_data$dominant_type[-1]) | is.na(smooth_data$dominant_type[-nrow(smooth_data)])))
  
  # Remove rows with NA dominant_type
  smooth_data <- smooth_data[!is.na(smooth_data$dominant_type), ]
  
  # Set up colors
  major_cell_types <- levels(plot_data$cell_type_plot)[levels(plot_data$cell_type_plot) != paste0("Other (< ", min_cells, " cells)")]
  
  if (is.null(color_palette)) {
    colors <- rainbow(length(major_cell_types))
    names(colors) <- major_cell_types
  } else {
    # Check if color_palette is a named vector
    if (is.null(names(color_palette))) {
      # Unnamed vector - assign colors sequentially
      if (length(color_palette) < length(major_cell_types)) {
        warning("Not enough colors in palette. Recycling colors.")
        colors <- rep(color_palette, length.out = length(major_cell_types))
      } else {
        colors <- color_palette[1:length(major_cell_types)]
      }
      names(colors) <- major_cell_types
    } else {
      # Named vector - match cell types to their specific colors
      colors <- character(length(major_cell_types))
      names(colors) <- major_cell_types
      
      # Assign colors for matched cell types
      matched_types <- intersect(major_cell_types, names(color_palette))
      colors[matched_types] <- color_palette[matched_types]
      
      # Assign colors for unmatched cell types
      unmatched_types <- setdiff(major_cell_types, names(color_palette))
      if (length(unmatched_types) > 0) {
        # Use unused colors from the palette
        used_colors <- color_palette[matched_types]
        unused_colors <- setdiff(color_palette, used_colors)
        
        if (length(unused_colors) >= length(unmatched_types)) {
          colors[unmatched_types] <- unused_colors[1:length(unmatched_types)]
        } else {
          # Not enough unused colors, use remaining colors and recycle if needed
          all_remaining <- c(unused_colors, color_palette)
          colors[unmatched_types] <- all_remaining[1:length(unmatched_types)]
        }
        
        message(paste("Warning: Cell types not found in color palette, assigned available colors:"))
        for (i in 1:length(unmatched_types)) {
          message(paste("  ", unmatched_types[i], "->", colors[unmatched_types[i]]))
        }
      }
    }
  }
  
  message(paste("Final color assignment for major cell types:"))
  for (i in 1:length(major_cell_types)) {
    message(paste("  ", i, ":", major_cell_types[i], "->", colors[major_cell_types[i]]))
  }
  
  message("=== DETERMINING LINEAGE ENDPOINT ===")
  
  # Determine lineage endpoint
  endpoint_name <- NULL
  if (is.null(lineage_destination)) {
    # Try to automatically determine endpoint
    endpoint_name <- get_lineage_endpoint(curves_obj, lineage_name, seurat_obj, cell_type_column)
    if (!is.null(endpoint_name)) {
      message(paste("✓ Automatically detected endpoint:", endpoint_name))
    } else {
      message("✗ Could not automatically determine endpoint")
    }
  } else if (lineage_destination != FALSE) {
    # Use provided endpoint
    endpoint_name <- lineage_destination
    message(paste("✓ Using provided endpoint:", endpoint_name))
  } else {
    # Explicitly disabled endpoint detection
    message("✓ Endpoint detection disabled")
  }
  
  # Create plot title with optional destination
  plot_title <- paste(gene_name, "Expression Trend Along", lineage_name)
  if (!is.null(endpoint_name)) {
    plot_title <- paste(gene_name, "Expression Trend Along", lineage_name, "-", endpoint_name)
  }
  
  # Create plot
  p <- ggplot() +
    # Add confidence interval
    geom_smooth(data = plot_data, aes(x = pseudotime, y = expression),
                method = "gam", formula = y ~ s(x, bs = "cs"), 
                se = TRUE, color = NA, fill = "darkgrey", alpha = 0.5) +
    # Add colored line segments
    geom_path(data = smooth_data, aes(x = pseudotime, y = expression, 
                                      color = dominant_type, group = segment_group), 
              linewidth = 2) +
    scale_color_manual(values = colors, name = "Dominant Cell Type") +
    labs(
      title = plot_title,
      x = "Pseudotime",
      y = paste(gene_name, "Expression"),
      subtitle = if (show_subtitle) {
        paste0("Line colored by dominant cell type in each pseudotime region (n = ", 
               nrow(plot_data), " cells, min_cells = ", min_cells, ", bins = ", n_bins, ")")
      } else {
        NULL
      }
    ) +
    theme_classic() +
    theme(
      plot.title = element_text(size = 14, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 11, hjust = 0.5),
      axis.title = element_text(size = 12),
      axis.text = element_text(size = 10),
      legend.title = element_text(size = 11),
      legend.text = element_text(size = 9),
      panel.grid.major.y = element_line(color = "grey90", linewidth = 0.5)
    ) +
    guides(color = guide_legend(override.aes = list(linewidth = 3)))
  
  # Add custom y-axis limits if provided
  if (!is.null(ylim)) {
    if (length(ylim) != 2 || !is.numeric(ylim)) {
      warning("ylim should be a numeric vector of length 2. Using automatic limits.")
    } else {
      p <- p + coord_cartesian(ylim = ylim)
    }
  }
  
  # Save plot if requested
  if (save_plot) {
    if (is.null(filename)) {
      filename <- paste0(gene_name, "_", lineage_name, "_pseudotime_trend.pdf")
    }
    ggsave(filename, plot = p, width = plot_width, height = plot_height)
    message(paste("Plot saved as:", filename))
  }
  
  return(p)
}

#' # Remove subtitle for cleaner plots
#' p <- plot_gene_pseudotime(df, curves_df_old_umap, "Atf3", "Lineage2",
#'                          show_subtitle = FALSE)
#' 
#' # Clean plot for publication with custom colors and no subtitle
#' p <- plot_gene_pseudotime(df, curves_df_old_umap, "Atf3", "Lineage2",
#'                          color_palette = atlas_color_palette,
#'                          show_subtitle = FALSE,
#'                          lineage_destination = FALSE)
#' 
#' # Complete usage example with subtitle control
#' p <- plot_gene_pseudotime(
#'   seurat_obj = df,                                    
#'   curves_obj = curves_df_old_umap,                   
#'   gene_name = "Atf3",                                
#'   lineage_name = "Lineage2",                         
#'   min_cells = 100,                                   
#'   n_bins = 30,                                       
#'   color_palette = r3dcol$cols_94,                    
#'   cell_type_column = "celltype_PijuanSala2019",      
#'   expression_slot = "data",
#'   exclude_cell_types = c("ExE endoderm", "Visceral endoderm"),
#'   cell_type_order = c("Epiblast", "Primitive Streak"),
#'   ylim = c(0, 4),                                    
#'   lineage_destination = "Neural Crest",
#'   show_subtitle = FALSE,                             # NEW: Remove subtitle
#'   save_plot = TRUE,                                  
#'   filename = "Atf3_Lineage2_clean.pdf",      
#'   plot_width = 12,                                   
#'   plot_height = 8                                    
#' )

### Color match

In [ ]:
# Color palette
r3dcol_cols_94 <- c(
  '#F0E442', '#1CE6FF', '#FF34FF', '#FF4A46', '#008941', '#006FA6', '#A30059', '#FFDBE5',
  '#7A4900', '#0089A3', '#63FFAC', '#B79762', '#004D43', '#8FB0FF', '#997D87', '#809693',
  '#6A3A4C', '#b4b701', '#4FC601', '#3B5DFF', '#FF2F80', '#61615A', '#BA0900', '#6B7900',
  '#00C2A0', '#FFAA92', '#FF90C9', '#B903AA', '#D16100', '#922329', '#7B4F4B', '#A1C299',
  '#0AA6D8', '#00846F', '#FFB500', '#C2FFED', '#A079BF', '#CC0744', '#C0B9B2', '#C2FF99',
  '#00489C', '#6F0062', '#0CBD66', '#EEC3FF', '#456D75', '#B77B68', '#7A87A1', '#788D66',
  '#885578', '#FAD09F', '#FF8A9A', '#D157A0', '#BEC459', '#456648', '#0086ED', '#886F4C',
  '#34362D', '#B4A8BD', '#00A6AA', '#452C2C', '#636375', '#A3C8C9', '#FF913F', '#938A81',
  '#575329', '#00FECF', '#B05B6F', '#8CD0FF', '#3B9700', '#04F757', '#C8A1A1', '#1E6E00',
  '#7900D7', '#A77500', '#6367A9', '#A05837', '#6B002C', '#772600', '#D790FF', '#9B9700',
  '#549E79', '#FFF69F', '#201625', '#72418F', '#BC23FF', '#99ADC0', '#3A2465', '#DDEFFF',
  '#5B4534', '#FDE8DC', '#404E55', '#CB7E98', '#A4E804', '#324E72'
)

# Dataset 1: atlas$celltype_extended_atlas (excluding neural/neuroectodermal types)
atlas_celltypes <- c(
  'Epiblast', 'Primitive Streak', 'ExE ectoderm', 'Visceral endoderm', 'ExE endoderm',
  'Non-neural ectoderm', 'Nascent mesoderm', 'Ectoderm', 'Blood progenitors', 'Paraxial mesoderm',
  'Caudal epiblast', 'Lateral plate mesoderm', 'Intermediate mesoderm', 'Cardiopharyngeal progenitors FHF',
  'PGC', 'Mesenchyme', 'Haematoendothelial progenitors', 'Gut tube', 'Cardiomyocytes FHF 1',
  'Allantois endothelium', 'Node', 'Erythroid', 'Embryo proper endothelium', 'Anterior somitic tissues',
  'Endotome', 'Amniotic ectoderm', 'Parietal endoderm', 'Anterior Primitive Streak', 'Presomitic mesoderm',
  'Allantois', 'Limb ectoderm', 'EMP', 'Anterior cardiopharyngeal progenitors', 'Pharyngeal mesoderm',
  'Limb mesoderm', 'Midgut', 'Venous endothelium', 'Pharyngeal endoderm', 'Hindgut', 'Notochord',
  'Epicardium', 'Posterior somitic tissues', 'Somitic mesoderm', 'Caudal mesoderm', 'Thyroid primordium',
  'Dermomyotome', 'Cardiomyocytes SHF 1', 'NMPs', 'Cardiomyocytes FHF 2', 'Cardiopharyngeal progenitors SHF',
  'Foregut', 'Surface ectoderm', 'YS mesothelium', 'Embryo proper mesothelium', 'Sclerotome',
  'Cardiomyocytes SHF 2', 'YS endothelium', 'NMPs/Mesoderm-biased', 'Kidney primordium', 'Endocardium',
  'MEP', 'Epidermis', 'Cranial mesoderm', 'Megakaryocyte progenitors', 'Cardiopharyngeal progenitors',
  'Chorioallantoic-derived erythroid progenitors', 'Frontonasal mesenchyme', 'YS mesothelium-derived endothelial progenitors',
  'Forelimb'
)

# Dataset 2: obj$new_subclusters_x
obj_celltypes <- c(
  'Naive Pluripotency', 'Epicardium', 'Limb Mesoderm', 'Allantois', 'EMP', 'Primitive Streak',
  'Mesenchyme', 'Nascent Mesoderm', 'Erythroid', 'Endocardium', 'Lateral Plate Mesoderm', 'PGC',
  'Epiblast', 'Amniotic Ectoderm', 'HEP/BP', 'Endothelium', 'Anterior Primitive Streak', 'MkP',
  'MEP', 'Visceral Endoderm', 'Atf3-KO Enriched'
)

# Define exact matches between datasets (same biological cell types get same color)
cell_type_matches <- list(
  # Direct matches (identical names or clear equivalents)
  "Epiblast" = list(atlas = "Epiblast", obj = "Epiblast", color = r3dcol_cols_94[1]),
  "Primitive Streak" = list(atlas = "Primitive Streak", obj = "Primitive Streak", color = r3dcol_cols_94[4]),
  "Anterior Primitive Streak" = list(atlas = "Anterior Primitive Streak", obj = "Anterior Primitive Streak", color = r3dcol_cols_94[29]),
  "Nascent Mesoderm" = list(atlas = "Nascent mesoderm", obj = "Nascent Mesoderm", color = r3dcol_cols_94[5]),  # Keep #008941 (dark teal)
  "Lateral Plate Mesoderm" = list(atlas = "Lateral plate mesoderm", obj = "Lateral Plate Mesoderm", color = r3dcol_cols_94[74]), # Changed to #A77500 (golden brown)
  "Limb Mesoderm" = list(atlas = "Limb mesoderm", obj = "Limb Mesoderm", color = r3dcol_cols_94[52]), # Changed to #D157A0 (magenta/purple)
  "PGC" = list(atlas = "PGC", obj = "PGC", color = r3dcol_cols_94[70]),
  "Mesenchyme" = list(atlas = "Mesenchyme", obj = "Mesenchyme", color = r3dcol_cols_94[15]),
  "Erythroid" = list(atlas = "Erythroid", obj = "Erythroid", color = r3dcol_cols_94[23]),
  "EMP" = list(atlas = "EMP", obj = "EMP", color = r3dcol_cols_94[30]),
  "MEP" = list(atlas = "MEP", obj = "MEP", color = r3dcol_cols_94[38]),
  "Epicardium" = list(atlas = "Epicardium", obj = "Epicardium", color = r3dcol_cols_94[3]),
  "Endocardium" = list(atlas = "Endocardium", obj = "Endocardium", color = r3dcol_cols_94[28]),
  "Allantois" = list(atlas = "Allantois", obj = "Allantois", color = r3dcol_cols_94[35]), # Changed from 11 to 35 (bright orange)
  "Visceral Endoderm" = list(atlas = "Visceral endoderm", obj = "Visceral Endoderm", color = r3dcol_cols_94[9]),
  "Amniotic Ectoderm" = list(atlas = "Amniotic ectoderm", obj = "Amniotic Ectoderm", color = r3dcol_cols_94[8]),
  
  # Functional matches (same biological function, different names)
  "Hematopoietic Progenitors" = list(atlas = c("Blood progenitors", "Haematoendothelial progenitors"), 
                                    obj = "HEP/BP", color = r3dcol_cols_94[76]),
  "Megakaryocyte Progenitors" = list(atlas = "Megakaryocyte progenitors", obj = "MkP", color = r3dcol_cols_94[78]),
  "Endothelium" = list(atlas = c("Embryo proper endothelium", "Venous endothelium", "YS endothelium", 
                                "Allantois endothelium", "YS mesothelium-derived endothelial progenitors"), 
                      obj = "Endothelium", color = r3dcol_cols_94[2])
)

# Create color vectors for both datasets
create_matched_color_vectors <- function(atlas_types, obj_types, matches, color_palette) {
  
  # Initialize color vectors
  atlas_colors <- character(length(atlas_types))
  names(atlas_colors) <- atlas_types
  
  obj_colors <- character(length(obj_types))
  names(obj_colors) <- obj_types
  
  # Track used colors
  used_colors <- character(0)
  
  # Assign matched colors first
  for (match_name in names(matches)) {
    match_info <- matches[[match_name]]
    color <- match_info$color
    used_colors <- c(used_colors, color)
    
    # Assign to atlas types
    if ("atlas" %in% names(match_info)) {
      atlas_matches <- intersect(match_info$atlas, atlas_types)
      if (length(atlas_matches) > 0) {
        atlas_colors[atlas_matches] <- color
      }
    }
    
    # Assign to obj types  
    if ("obj" %in% names(match_info)) {
      obj_matches <- intersect(match_info$obj, obj_types)
      if (length(obj_matches) > 0) {
        obj_colors[obj_matches] <- color
      }
    }
  }
  
  # Get remaining colors for unmatched cell types
  remaining_colors <- setdiff(color_palette, used_colors)
  color_index <- 1
  
  # Assign colors to unmatched atlas cell types
  atlas_unmatched <- names(atlas_colors)[atlas_colors == ""]
  for (cell_type in atlas_unmatched) {
    if (color_index <= length(remaining_colors)) {
      atlas_colors[cell_type] <- remaining_colors[color_index]
      color_index <- color_index + 1
    }
  }
  
  # Assign colors to unmatched obj cell types
  obj_unmatched <- names(obj_colors)[obj_colors == ""]
  for (cell_type in obj_unmatched) {
    if (color_index <= length(remaining_colors)) {
      obj_colors[cell_type] <- remaining_colors[color_index]
      color_index <- color_index + 1
    }
  }
  
  return(list(atlas = atlas_colors, obj = obj_colors))
}

# Generate the color mappings
color_mappings <- create_matched_color_vectors(atlas_celltypes, obj_celltypes, 
                                             cell_type_matches, r3dcol_cols_94)

# Extract the final color vectors
atlas_color_palette <- color_mappings$atlas
obj_color_palette <- color_mappings$obj

# Print the results
cat("=== ATLAS DATASET COLOR MAPPING ===\n")
for (i in 1:length(atlas_color_palette)) {
  cat(sprintf("%-40s -> %s\n", names(atlas_color_palette)[i], atlas_color_palette[i]))
}

cat("\n=== OBJ DATASET COLOR MAPPING ===\n")
for (i in 1:length(obj_color_palette)) {
  cat(sprintf("%-25s -> %s\n", names(obj_color_palette)[i], obj_color_palette[i]))
}

cat("\n=== MATCHED CELL TYPES (SAME COLORS) ===\n")
for (match_name in names(cell_type_matches)) {
  match_info <- cell_type_matches[[match_name]]
  cat(sprintf("%-25s: %s\n", match_name, match_info$color))
  if (length(match_info$atlas) == 1) {
    cat(sprintf("  Atlas: %s\n", match_info$atlas))
  } else {
    cat(sprintf("  Atlas: %s\n", paste(match_info$atlas, collapse = ", ")))
  }
  cat(sprintf("  Obj:   %s\n\n", match_info$obj))
}

# Usage in your plotting function:
cat("\n=== USAGE EXAMPLES ===\n")
cat("# For atlas dataset:\n")
cat("p1 <- plot_gene_pseudotime(atlas_seurat, curves_obj, 'Atf3', 'Lineage1',\n")
cat("                           color_palette = atlas_color_palette)\n\n")
cat("# For obj dataset:\n") 
cat("p2 <- plot_gene_pseudotime(obj_seurat, curves_obj, 'Atf3', 'Lineage1',\n")
cat("                           color_palette = obj_color_palette)\n\n")
cat("# Now p1 and p2 will have matching colors for equivalent cell types!\n")

In [ ]:
r3dcol_cols_94

# Atf3 - LM

## in vitro lm

In [ ]:
cmo_lm_xobj <- readRDS('./cmo_v3_latemesoderm_lineage.rds')
curves_cmo_lm <- readRDS('./cmo_v3_latemesoderm_slingshot_curves.rds')

In [ ]:
curves_cmo_lm_mesenchyme

In [ ]:
p_Atf3_cmo_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_mesenchyme_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_mesenchyme,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_limb_mesoderm

In [ ]:
p_Atf3_cmo_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_limb_mesoderm_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_limb_mesoderm,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
 
 exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_allantois

In [ ]:
p_Atf3_cmo_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_allantois_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_allantois,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_epicardium

In [ ]:
p_Atf3_cmo_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_epicardium_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_epicardium,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Atf3_cmo_lm_epicardium)
print(p_Atf3_cmo_lm_allantois)
print(p_Atf3_cmo_lm_limb_mesoderm)
print(p_Atf3_cmo_lm_mesenchyme)

## in vivo lm

In [ ]:
# after crush
atlas_lm_xobj <- readRDS('./atlas_latemesoderm_lineage.rds') # it should be v4 actually.
lineages_atlas_lm <- readRDS('./atlas_late_Mesoderm_slingshot_lin.rds')
curves_atlas_lm <- readRDS('./atlas_late_mesoderm_slingshot_curves.rds')

In [ ]:
atlas_lm_xobj

In [ ]:
curves_atlas_lm

In [ ]:
p_Atf3_atlas_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Atf3_atlas_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Atf3_atlas_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                  # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage2",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Atf3_atlas_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Atf3_atlas_lm_epicardium)
print(p_Atf3_atlas_lm_allantois)
print(p_Atf3_atlas_lm_limb_mesoderm)
print(p_Atf3_atlas_lm_mesenchyme)

## lM

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 9)
plot_grid(
    p_Atf3_atlas_lm_allantois,p_Atf3_atlas_lm_epicardium, p_Atf3_atlas_lm_limb_mesoderm, p_Atf3_atlas_lm_mesenchyme,
    p_Atf3_cmo_lm_allantois, p_Atf3_cmo_lm_epicardium, p_Atf3_cmo_lm_limb_mesoderm, p_Atf3_cmo_lm_mesenchyme,
    ncol =  4
)

In [ ]:
obj <- readRDS('../../cmo_version4.rds')
obj

In [ ]:
DotPlot(obj, features = 'Tbx4', group.by = 'new_subclusters_x' )

# Zfp711 - LM

## in vitro lm

In [ ]:
p_Zfp711_cmo_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_mesenchyme_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_mesenchyme,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_limb_mesoderm

In [ ]:
p_Zfp711_cmo_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_limb_mesoderm_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_limb_mesoderm,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
 
 exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_allantois

In [ ]:
p_Zfp711_cmo_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_allantois_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_allantois,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_epicardium

In [ ]:
p_Zfp711_cmo_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_epicardium_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_epicardium,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Zfp711_cmo_lm_epicardium)
print(p_Zfp711_cmo_lm_allantois)
print(p_Zfp711_cmo_lm_limb_mesoderm)
print(p_Zfp711_cmo_lm_mesenchyme)

## in vivo lm

In [ ]:
# after crush
atlas_lm_xobj <- readRDS('./atlas_latemesoderm_lineage.rds') # it should be v4 actually.
lineages_atlas_lm <- readRDS('./atlas_late_Mesoderm_slingshot_lin.rds')
curves_atlas_lm <- readRDS('./atlas_late_mesoderm_slingshot_curves.rds')

In [ ]:
atlas_lm_xobj

In [ ]:
curves_atlas_lm

In [ ]:
p_Zfp711_atlas_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_atlas_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_atlas_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                  # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage2",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_atlas_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Zfp711_atlas_lm_epicardium)
print(p_Zfp711_atlas_lm_allantois)
print(p_Zfp711_atlas_lm_limb_mesoderm)
print(p_Zfp711_atlas_lm_mesenchyme)

## lM

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 9)
plot_grid(
    p_Zfp711_atlas_lm_allantois,p_Zfp711_atlas_lm_epicardium, p_Zfp711_atlas_lm_limb_mesoderm, p_Zfp711_atlas_lm_mesenchyme,
    p_Zfp711_cmo_lm_allantois, p_Zfp711_cmo_lm_epicardium, p_Zfp711_cmo_lm_limb_mesoderm, p_Zfp711_cmo_lm_mesenchyme,
    ncol =  4
)

# Bcl6b - LM

## in vitro lm

In [ ]:
curves_cmo_lm_mesenchyme

In [ ]:
p_Bcl6b_cmo_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_mesenchyme_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_mesenchyme,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_limb_mesoderm

In [ ]:
p_Bcl6b_cmo_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_limb_mesoderm_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_limb_mesoderm,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
 
 exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_allantois

In [ ]:
p_Bcl6b_cmo_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_allantois_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_allantois,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
curves_cmo_lm_epicardium

In [ ]:
p_Bcl6b_cmo_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_lm_epicardium_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_lm_epicardium,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 1.2),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
    
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Bcl6b_cmo_lm_epicardium)
print(p_Bcl6b_cmo_lm_allantois)
print(p_Bcl6b_cmo_lm_limb_mesoderm)
print(p_Bcl6b_cmo_lm_mesenchyme)

## in vivo lm

In [ ]:
# after crush
atlas_lm_xobj <- readRDS('./atlas_latemesoderm_lineage.rds') # it should be v4 actually.
lineages_atlas_lm <- readRDS('./atlas_late_Mesoderm_slingshot_lin.rds')
curves_atlas_lm <- readRDS('./atlas_late_mesoderm_slingshot_curves.rds')

In [ ]:
atlas_lm_xobj

In [ ]:
curves_atlas_lm

In [ ]:
p_Bcl6b_atlas_lm_mesenchyme <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_atlas_lm_limb_mesoderm <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_atlas_lm_allantois <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                  # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage2",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_atlas_lm_epicardium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_lm_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_lm,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Bcl6b_atlas_lm_epicardium)
print(p_Bcl6b_atlas_lm_allantois)
print(p_Bcl6b_atlas_lm_limb_mesoderm)
print(p_Bcl6b_atlas_lm_mesenchyme)

## lM

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 9)
plot_grid(
    p_Bcl6b_atlas_lm_allantois,p_Bcl6b_atlas_lm_epicardium, p_Bcl6b_atlas_lm_limb_mesoderm, p_Bcl6b_atlas_lm_mesenchyme,
    p_Bcl6b_cmo_lm_allantois, p_Bcl6b_cmo_lm_epicardium, p_Bcl6b_cmo_lm_limb_mesoderm, p_Bcl6b_cmo_lm_mesenchyme,
    ncol =  4
)

# Atf3 -Blood

## in vitro lm

In [ ]:
cmo_hep_xobj <- readRDS('./cmo_v3_HEP_lineage.rds')
curves_cmo_hep <- readRDS('./cmo_v3_hep_slingshot_curves.rds')

In [ ]:
curves_cmo_hep

In [ ]:
p_Atf3_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 2.25),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Atf3_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 2.25),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Atf3_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 2.25),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Atf3_emp)
print(p_Atf3_endothelium)
print(p_Atf3_erythroid)

## in vivo lm

In [ ]:
# after crush
atlas_emp_xobj <- readRDS('./atlas_emp_lineage.rds') # it should be v4 actually.
curves_atlas_emp <- readRDS('./atlas_emp_slingshot_curves.rds')

In [ ]:
curves_atlas_emp

In [ ]:
p_Atf3_atlas_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_emp_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_emp,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_endothelium_xobj <- readRDS('./atlas_endothelium_lineage.rds') # it should be v4 actually.
curves_atlas_endothelium <- readRDS('./atlas_endothelium_slingshot_curves.rds')

In [ ]:
curves_atlas_endothelium

In [ ]:
p_Atf3_atlas_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_endothelium_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_endothelium,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_erythroid_xobj <- readRDS('./atlas_erythroid_lineage.rds') # it should be v4 actually.
curves_atlas_erythroid <- readRDS('./atlas_erythroid_slingshot_curves.rds')

In [ ]:
curves_atlas_erythroid

In [ ]:
p_Atf3_atlas_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_erythroid_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_erythroid,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Atf3_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Atf3_atlas_endothelium)
print(p_Atf3_atlas_emp)
print(p_Atf3_atlas_erythroid)

## lM

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 9)
plot_grid(
    p_Atf3_atlas_endothelium,p_Atf3_atlas_erythroid, p_Atf3_atlas_emp,
    p_Atf3_endothelium, p_Atf3_erythroid, p_Atf3_emp, 
    ncol =  3
)

# Zfp711 -Blood

## in vitro lm

In [ ]:
cmo_hep_xobj <- readRDS('./cmo_v3_HEP_lineage.rds')
curves_cmo_hep <- readRDS('./cmo_v3_hep_slingshot_curves.rds')

In [ ]:
curves_cmo_hep

In [ ]:
p_Zfp711_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Zfp711_emp)
print(p_Zfp711_endothelium)
print(p_Zfp711_erythroid)

## in vivo lm

In [ ]:
# after crush
atlas_emp_xobj <- readRDS('./atlas_emp_lineage.rds') # it should be v4 actually.
curves_atlas_emp <- readRDS('./atlas_emp_slingshot_curves.rds')

In [ ]:
curves_atlas_emp

In [ ]:
p_Zfp711_atlas_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_emp_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_emp,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 1.6),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_endothelium_xobj <- readRDS('./atlas_endothelium_lineage.rds') # it should be v4 actually.
curves_atlas_endothelium <- readRDS('./atlas_endothelium_slingshot_curves.rds')

In [ ]:
curves_atlas_endothelium

In [ ]:
p_Zfp711_atlas_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_endothelium_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_endothelium,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 1.6),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_erythroid_xobj <- readRDS('./atlas_erythroid_lineage.rds') # it should be v4 actually.
curves_atlas_erythroid <- readRDS('./atlas_erythroid_slingshot_curves.rds')

In [ ]:
curves_atlas_erythroid

In [ ]:
p_Zfp711_atlas_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_erythroid_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_erythroid,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 1.6),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Zfp711_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Zfp711_atlas_endothelium)
print(p_Zfp711_atlas_emp)
print(p_Zfp711_atlas_erythroid)

## lM

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 9)
plot_grid(
    p_Zfp711_atlas_endothelium,p_Zfp711_atlas_erythroid, p_Zfp711_atlas_emp,
    p_Zfp711_endothelium, p_Zfp711_erythroid, p_Zfp711_emp, 
    ncol =  3
)

# Bcl6b -Blood

## in vitro lm

In [ ]:
cmo_hep_xobj <- readRDS('./cmo_v3_HEP_lineage.rds')
curves_cmo_hep <- readRDS('./cmo_v3_hep_slingshot_curves.rds')

In [ ]:
curves_cmo_hep

In [ ]:
p_Bcl6b_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage4",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage3",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = cmo_hep_xobj,                                    # Your Seurat object
  curves_obj = curves_cmo_hep,                   # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "new_subclusters_x",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = obj_color_palette,                    # Custom color palette
  ylim = c(0, 0.5),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint

  exclude_cell_types = c("Naive Pluripotency"),
 # cell_type_order = c("Epiblast", "Primitive Streak"),
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Bcl6b_emp)
print(p_Bcl6b_endothelium)
print(p_Bcl6b_erythroid)

## in vivo lm

In [ ]:
# after crush
atlas_emp_xobj <- readRDS('./atlas_emp_lineage.rds') # it should be v4 actually.
curves_atlas_emp <- readRDS('./atlas_emp_slingshot_curves.rds')

In [ ]:
curves_atlas_emp

In [ ]:
p_Bcl6b_atlas_emp <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_emp_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_emp,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_endothelium_xobj <- readRDS('./atlas_endothelium_lineage.rds') # it should be v4 actually.
curves_atlas_endothelium <- readRDS('./atlas_endothelium_slingshot_curves.rds')

In [ ]:
curves_atlas_endothelium

In [ ]:
p_Bcl6b_atlas_endothelium <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_endothelium_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_endothelium,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
# after crush
atlas_erythroid_xobj <- readRDS('./atlas_erythroid_lineage.rds') # it should be v4 actually.
curves_atlas_erythroid <- readRDS('./atlas_erythroid_slingshot_curves.rds')

In [ ]:
curves_atlas_erythroid

In [ ]:
p_Bcl6b_atlas_erythroid <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_erythroid_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_erythroid,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.9),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
print(p_Bcl6b_atlas_endothelium)
print(p_Bcl6b_atlas_emp)
print(p_Bcl6b_atlas_erythroid)

## lM

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 9)
plot_grid(
    p_Bcl6b_atlas_endothelium,p_Bcl6b_atlas_erythroid, p_Bcl6b_atlas_emp,
    p_Bcl6b_endothelium, p_Bcl6b_erythroid, p_Bcl6b_emp, 
    ncol =  3
)

# Cardiac

## FHF

In [ ]:
# after crush
atlas_fhf_xobj <- readRDS('./atlas_cardiac_fhf_lineage.rds') # it should be v4 actually.
curves_atlas_fhf <- readRDS('./atlas_cardiac_fhf_slingshot_curves.rds')

In [ ]:
curves_atlas_fhf

In [ ]:
p_Atf3_atlas_fhf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_fhf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_fhf,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_atlas_fhf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_fhf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_fhf,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_atlas_fhf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_fhf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_fhf,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

## SHF

In [ ]:
# after crush
atlas_shf_xobj <- readRDS('./atlas_cardiac_shf_lineage.rds') # it should be v4 actually.
curves_atlas_shf <- readRDS('./atlas_cardiac_shf_slingshot_curves.rds')

In [ ]:
curves_atlas_shf

In [ ]:
p_Atf3_atlas_shf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_shf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_shf,                 # Your slingshot curves object  
  gene_name = "Atf3",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Zfp711_atlas_shf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_shf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_shf,                 # Your slingshot curves object  
  gene_name = "Zfp711",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

In [ ]:
p_Bcl6b_atlas_shf <- plot_gene_pseudotime(
  # === REQUIRED PARAMETERS ===
  seurat_obj = atlas_shf_xobj,                                    # Your Seurat object
  curves_obj = curves_atlas_shf,                 # Your slingshot curves object  
  gene_name = "Bcl6b",                                # Gene to plot
  lineage_name = "Lineage1",                         # Which lineage trajectory
  
  # === ANALYSIS PARAMETERS ===
  min_cells = 100,                                   # Minimum cells for major cell types
  n_bins = 200,                                       # Number of pseudotime bins
  cell_type_column = "celltype_extended_atlas",      # Cell type annotation column
  expression_slot = "data",                          # Expression data slot ("data", "counts", "scale.data")
  
  # === VISUALIZATION PARAMETERS ===
  color_palette = atlas_color_palette,                    # Custom color palette
  ylim = c(0, 0.75),                                    # Y-axis limits [min, max]
 # lineage_destination = "Neural Crest",              # Override auto-detected endpoint
        show_subtitle = FALSE,
  
  # === OUTPUT PARAMETERS ===
  save_plot = FALSE,                                  # Save the plot automatically
  filename = "Bcl6b_Lineage2_neural_crest.pdf",      # Custom filename  
  plot_width = 12,                                   # Plot width in inches
  plot_height = 8                                    # Plot height in inches
)

## IM

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 4.5)
plot_grid(
p_Atf3_atlas_fhf,p_Zfp711_atlas_fhf,p_Bcl6b_atlas_fhf,
    ncol=3
)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 4.5)
plot_grid(
    p_Atf3_atlas_shf,p_Zfp711_atlas_shf,p_Bcl6b_atlas_shf, 
    ncol=3
)

# Plot - Atf3

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_atf3_1 <- 
plot_grid(
    p_Atf3_atlas_lm_allantois,p_Atf3_atlas_lm_epicardium, p_Atf3_atlas_lm_limb_mesoderm,
    p_Atf3_cmo_lm_allantois, p_Atf3_cmo_lm_epicardium, p_Atf3_cmo_lm_limb_mesoderm,
    ncol =  3
)
p_atf3_1

In [ ]:
options(repr.plot.width = 8.3, repr.plot.height = 9)
p_atf3_2 <-
plot_grid(
    p_Atf3_atlas_lm_limb_mesoderm,
    p_Atf3_cmo_lm_limb_mesoderm,
    ncol =  1
)
p_atf3_2

In [ ]:

options(repr.plot.width = 25, repr.plot.height = 9)
p_atf3_3 <- plot_grid(
    p_Atf3_atlas_endothelium,p_Atf3_atlas_erythroid, p_Atf3_atlas_emp,
    p_Atf3_endothelium, p_Atf3_erythroid, p_Atf3_emp, 
    ncol =  3
)
p_atf3_3

In [ ]:
options(repr.plot.width = 16.6, repr.plot.height = 4.5)
p_atf3_4 <-
plot_grid(
    p_Atf3_atlas_fhf,
    p_Atf3_atlas_shf,
    ncol =  2
)
p_atf3_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_atf3_2_4 <-
plot_grid(
    p_Atf3_atlas_lm_limb_mesoderm,p_Atf3_atlas_fhf,p_Atf3_atlas_shf,
    p_Atf3_cmo_lm_limb_mesoderm, NULL, NULL,
    ncol =  3
)
p_atf3_2_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 27)

p_atf3_combined <-
plot_grid(
    p_atf3_3,p_atf3_1,p_atf3_2_4,

    ncol =  1
)
p_atf3_combined

In [ ]:
pdf(file = './figures_3/atf3_pseudotime_combined.pdf',width = 25, height = 27)
p_atf3_combined
dev.off()

In [ ]:
png(file = './figures_3/atf3_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_atf3_combined
dev.off()

In [ ]:
png(file = './figures_3/atf3_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_atf3_combined
dev.off()

In [ ]:
svg(file = './figures_3/atf3_pseudotime_combined.svg',width = 25, height = 27)
p_atf3_combined
dev.off()

# Plot-Zfp711

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Zfp711_1 <- 
plot_grid(
    p_Zfp711_atlas_lm_allantois,p_Zfp711_atlas_lm_epicardium, p_Zfp711_atlas_lm_limb_mesoderm,
    p_Zfp711_cmo_lm_allantois, p_Zfp711_cmo_lm_epicardium, p_Zfp711_cmo_lm_limb_mesoderm,
    ncol =  3
)
p_Zfp711_1

In [ ]:
options(repr.plot.width = 8.3, repr.plot.height = 9)
p_Zfp711_2 <-
plot_grid(
    p_Zfp711_atlas_lm_limb_mesoderm,
    p_Zfp711_cmo_lm_limb_mesoderm,
    ncol =  1
)
p_Zfp711_2

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 11)
p_second_wave <- plot_grid(
    p_dfa_emp_st_atf3,p_dfa_emp_st_zfp711,p_dfa_emp_st_bcl6b,
    p_dfc_emp_st_atf3,p_dfc_emp_st_zfp711,p_dfc_emp_st_bcl6b,
    
ncol = 3)
p_second_wave

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Zfp711_3 <- plot_grid(
    p_Zfp711_atlas_endothelium,p_Zfp711_atlas_erythroid, p_Zfp711_atlas_emp,
    p_Zfp711_endothelium, p_Zfp711_erythroid, p_Zfp711_emp, 
    ncol =  3
)
p_Zfp711_3

In [ ]:
options(repr.plot.width = 16.6, repr.plot.height = 4.5)
p_Zfp711_4 <-
plot_grid(
    p_Zfp711_atlas_fhf,
    p_Zfp711_atlas_shf,
    ncol =  2
)
p_Zfp711_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Zfp711_2_4 <-
plot_grid(
    p_Zfp711_atlas_lm_limb_mesoderm,p_Zfp711_atlas_fhf,p_Zfp711_atlas_shf,
    p_Zfp711_cmo_lm_limb_mesoderm, NULL, NULL,
    ncol =  3
)
p_Zfp711_2_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 27)

p_Zfp711_combined <-
plot_grid(
    p_Zfp711_3,p_Zfp711_1,p_Zfp711_2_4,

    ncol =  1
)
p_Zfp711_combined

In [ ]:
pdf(file = './figures_3/Zfp711_pseudotime_combined.pdf',width = 25, height = 27)
p_Zfp711_combined
dev.off()

In [ ]:
png(file = './figures_3/Zfp711_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_Zfp711_combined
dev.off()

In [ ]:
png(file = './figures_3/Zfp711_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_Zfp711_combined
dev.off()

In [ ]:
svg(file = './figures_3/Zfp711_pseudotime_combined.svg',width = 25, height = 27)
p_Zfp711_combined
dev.off()

# Plot- Bcl6b

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Bcl6b_1 <- 
plot_grid(
    p_Bcl6b_atlas_lm_allantois,p_Bcl6b_atlas_lm_epicardium, p_Bcl6b_atlas_lm_limb_mesoderm,
    p_Bcl6b_cmo_lm_allantois, p_Bcl6b_cmo_lm_epicardium, p_Bcl6b_cmo_lm_limb_mesoderm,
    ncol =  3
)
p_Bcl6b_1

In [ ]:
options(repr.plot.width = 8.3, repr.plot.height = 9)
p_Bcl6b_2 <-
plot_grid(
    p_Bcl6b_atlas_lm_limb_mesoderm,
    p_Bcl6b_cmo_lm_limb_mesoderm,
    ncol =  1
)
p_Bcl6b_2

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Bcl6b_3 <- plot_grid(
    p_Bcl6b_atlas_endothelium,p_Bcl6b_atlas_erythroid, p_Bcl6b_atlas_emp,
    p_Bcl6b_endothelium, p_Bcl6b_erythroid, p_Bcl6b_emp, 
    ncol =  3
)
p_Bcl6b_3

In [ ]:
options(repr.plot.width = 16.6, repr.plot.height = 4.5)
p_Bcl6b_4 <-
plot_grid(
    p_Bcl6b_atlas_fhf,
    p_Bcl6b_atlas_shf,
    ncol =  2
)
p_Bcl6b_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 9)
p_Bcl6b_2_4 <-
plot_grid(
    p_Bcl6b_atlas_lm_limb_mesoderm,p_Bcl6b_atlas_fhf,p_Bcl6b_atlas_shf,
    p_Bcl6b_cmo_lm_limb_mesoderm, NULL, NULL,
    ncol =  3
)
p_Bcl6b_2_4

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 27)

p_Bcl6b_combined <-
plot_grid(
    p_Bcl6b_3,p_Bcl6b_1,p_Bcl6b_2_4,

    ncol =  1
)
p_Bcl6b_combined

In [ ]:
pdf(file = './figures_3/Bcl6b_pseudotime_combined.pdf',width = 25, height = 27)
p_Bcl6b_combined
dev.off()

In [ ]:
png(file = './figures_3/Bcl6b_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_Bcl6b_combined
dev.off()

In [ ]:
png(file = './figures_3/Bcl6b_pseudotime_combined.png',width = 25, height = 27,units = 'in',res = 300)
p_Bcl6b_combined
dev.off()

In [ ]:
svg(file = './figures_3/Bcl6b_pseudotime_combined.svg',width = 25, height = 27)
p_Bcl6b_combined
dev.off()

# Figure different order

In [ ]:
   title = expression(paste(italic("In vitro"), " average ", italic("Bcl6b "), "expression: second wave")),

In [ ]:
expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: EMP Trajectory"))

In [ ]:
p_Atf3_atlas_endothelium + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: endothelium trajectory")),
                                   subtitle = NULL) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5))  # hjust = 0.5 centers the title

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_emp_combined <-
plot_grid(
    p_Atf3_atlas_emp + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: EMP trajectory"))),
    p_Zfp711_atlas_emp + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: EMP trajectory"))),
    p_Bcl6b_atlas_emp  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: EMP trajectory"))),
    p_Atf3_emp + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: EMP trajectory"))),
    p_Zfp711_emp + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: EMP trajectory"))),
    p_Bcl6b_emp + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: EMP trajectory"))),

    ncol =  3
)
p_emp_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_endothelium_combined <-
plot_grid(
    p_Atf3_atlas_endothelium + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: endothelium rajectory"))),
    p_Zfp711_atlas_endothelium + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: endothelium trajectory"))),
    p_Bcl6b_atlas_endothelium  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: endothelium trajectory"))),
    p_Atf3_endothelium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: endothelium trajectory"))),
    p_Zfp711_endothelium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: endothelium trajectory"))),
    p_Bcl6b_endothelium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: endothelium trajectory"))),

    ncol =  3
)
p_endothelium_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 16)
p_emp_endothelium <-
plot_grid(
    p_endothelium_combined,
    p_emp_combined,
    ncol= 1
)
p_emp_endothelium

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_erythroid_combined <-
plot_grid(
    p_Atf3_atlas_erythroid + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: erythroid trajectory"))),
    p_Zfp711_atlas_erythroid + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: erythroid trajectory"))),
    p_Bcl6b_atlas_erythroid  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: erythroid trajectory"))),
    p_Atf3_erythroid + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: erythroid trajectory"))),
    p_Zfp711_erythroid + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: erythroid trajectory"))),
    p_Bcl6b_erythroid + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: erythroid trajectory"))),

    ncol =  3
)
p_erythroid_combined

##### I am ready (pseudotime HEP lineages)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 24)
p_emp_endothelium_erythroid <-
plot_grid(
    p_endothelium_combined,
    p_emp_combined,
    p_erythroid_combined,
    ncol= 1
)
p_emp_endothelium_erythroid

In [ ]:
pdf(file = './figures_3/plot_pseudotime_hep_lineages.pdf', width = 30, height = 24)
p_emp_endothelium_erythroid
dev.off()

In [ ]:
svg(file = './figures_3/plot_pseudotime_hep_lineages.svg', width = 30, height = 24)
p_emp_endothelium_erythroid
dev.off()

In [ ]:
png(file = './figures_3/plot_pseudotime_hep_lineages.png', width = 30, height = 24, units = 'in',res = 300)
p_emp_endothelium_erythroid
dev.off()

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_allantois_combined <-
plot_grid(
    p_Atf3_atlas_lm_allantois + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: allantois trajectory"))),
    p_Zfp711_atlas_lm_allantois + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: allantois trajectory"))),
    p_Bcl6b_atlas_lm_allantois  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: allantois trajectory"))),
    p_Atf3_cmo_lm_allantois + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: allantois trajectory"))),
    p_Zfp711_cmo_lm_allantois + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: allantois trajectory"))),
    p_Bcl6b_cmo_lm_allantois + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: allantois trajectory"))),

    ncol =  3
)
p_allantois_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_epicardium_combined <-
plot_grid(
    p_Atf3_atlas_lm_epicardium + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: epicardium trajectory"))),
    p_Zfp711_atlas_lm_epicardium + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: epicardium trajectory"))),
    p_Bcl6b_atlas_lm_epicardium  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: epicardium trajectory"))),
    p_Atf3_cmo_lm_epicardium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: epicardium trajectory"))),
    p_Zfp711_cmo_lm_epicardium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: epicardium trajectory"))),
    p_Bcl6b_cmo_lm_epicardium + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: epicardium trajectory"))),

    ncol =  3
)
p_epicardium_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_mesenchyme_combined <-
plot_grid(
    p_Atf3_atlas_lm_mesenchyme + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: mesenchyme trajectory"))),
    p_Zfp711_atlas_lm_mesenchyme + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: mesenchyme trajectory"))),
    p_Bcl6b_atlas_lm_mesenchyme  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: mesenchyme trajectory"))),
    p_Atf3_cmo_lm_mesenchyme + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: mesenchyme trajectory"))),
    p_Zfp711_cmo_lm_mesenchyme + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: mesenchyme trajectory"))),
    p_Bcl6b_cmo_lm_mesenchyme + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: mesenchyme trajectory"))),

    ncol =  3
)
p_mesenchyme_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_limb_mesoderm_combined <-
plot_grid(
    p_Atf3_atlas_lm_limb_mesoderm + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: limb mesoderm trajectory"))),
    p_Zfp711_atlas_lm_limb_mesoderm + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: limb mesoderm trajectory"))),
    p_Bcl6b_atlas_lm_limb_mesoderm  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: limb mesoderm trajectory"))),
    p_Atf3_cmo_lm_limb_mesoderm + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: limb mesoderm trajectory"))),
    p_Zfp711_cmo_lm_limb_mesoderm + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: limb mesoderm trajectory"))),
    p_Bcl6b_cmo_lm_limb_mesoderm + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: limb mesoderm trajectory"))),

    ncol =  3
)
p_limb_mesoderm_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 8)

p_cardiac_combined <-
plot_grid(
    p_Atf3_atlas_fhf + ggtitle(expression(paste(italic("In vivo"), " ", italic("Atf3 "), "expression: first heart field trajectory"))),
    p_Zfp711_atlas_fhf + ggtitle(expression(paste(italic("In vivo"), " ", italic("Zfp711 "), "expression: first heart field trajectory"))),
    p_Bcl6b_atlas_fhf  + ggtitle(expression(paste(italic("In vivo"), " ", italic("Bcl6b "), "expression: first heart field trajectory"))),
    p_Atf3_atlas_shf + ggtitle(expression(paste(italic("In vitro"), " ", italic("Atf3 "), "expression: second heart field trajectory"))),
    p_Zfp711_atlas_shf + ggtitle(expression(paste(italic("In vitro"), " ", italic("Zfp711 "), "expression: second heart field trajectory"))),
    p_Bcl6b_atlas_shf + ggtitle(expression(paste(italic("In vitro"), " ", italic("Bcl6b "), "expression: second heart field trajectory"))),

    ncol =  3
)
p_cardiac_combined

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 24)
p_epicardium_allantois_mesenchyme <-
plot_grid(
    p_epicardium_combined,
    p_allantois_combined,
    p_mesenchyme_combined,
    ncol= 1
)
p_epicardium_allantois_mesenchyme

In [ ]:
pdf(file = './figures_3/plot_pseudotime_latemesoderm_lineages.pdf', width = 30, height = 24)
p_epicardium_allantois_mesenchyme
dev.off()

In [ ]:
svg(file = './figures_3/plot_pseudotime_latemesoderm_lineages.svg', width = 30, height = 24)
p_epicardium_allantois_mesenchyme
dev.off()

In [ ]:
png(file = './figures_3/plot_pseudotime_latemesoderm_lineages.png', width = 30, height = 24, units = 'in',res = 300)
p_epicardium_allantois_mesenchyme
dev.off()

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 16)
p_limbmesoderm_fhf_shf <-
plot_grid(
    p_limb_mesoderm_combined,
    p_cardiac_combined,
    ncol= 1
)
p_limbmesoderm_fhf_shf

In [ ]:
pdf(file = './figures_3/plot_pseudotime_latemesoderm_and_cardiac.pdf', width = 30, height = 16)
p_limbmesoderm_fhf_shf
dev.off()

In [ ]:
svg(file = './figures_3/plot_pseudotime_latemesoderm_and_cardiac.svg', width = 30, height = 16)
p_limbmesoderm_fhf_shf
dev.off()

In [ ]:
png(file = './figures_3/plot_pseudotime_latemesoderm_and_cardiac.png', width = 30, height = 16, units = 'in',res = 300)
p_limbmesoderm_fhf_shf
dev.off()

# Manual Pseudotime

In [ ]:
Atf3_paralogs <- c('Atf3','Jdp2','Batf3','Batf','Fosl1','Fosb','Fosl2')

In [ ]:
Zfp711_paralogs <- c('Zfp711','Zfy1','Zfy2','Zfx')

In [ ]:
Bcl6b_paralogs <-c('Bcl6b',"Bcl6")

In [ ]:
KOs_and_paralogs <- c(Atf3_paralogs,Zfp711_paralogs,Bcl6b_paralogs)

In [ ]:
ps_atlas <- AverageExpression(object = atlas,features = KOs_and_paralogs, group.by = 'celltype_extended_atlas', layer = 'data'  )

In [ ]:
head(ps_atlas)

In [ ]:
obj
wt_obj <- subset(obj, subset = Conditions_lv %in% "Control")
wt_obj

In [ ]:
wt_obj$new_subclusters_x

In [ ]:
ps_wt_obj_x <- AverageExpression(object = wt_obj,features = KOs_and_paralogs, group.by = 'new_subclusters_x', layer = 'data'  )

In [ ]:
ps_wt_obj <- AverageExpression(object = wt_obj,features = KOs_and_paralogs, group.by = 'new_subclusters', layer = 'data'  )

In [ ]:
colnames(ps_wt_obj$RNA)

In [ ]:
colnames(ps_wt_obj_x$RNA)

In [ ]:
colnames(ps_atlas$originalexp)

In [ ]:
dfa <-as.data.frame(ps_atlas$originalexp)
dfc  <- as.data.frame(ps_wt_obj$RNA)
dfcx  <-as.data.frame( ps_wt_obj_x$RNA)

## EMP

### in vivo

In [ ]:
curves_atlas_emp

In [ ]:
unique(atlas$celltype_extended_atlas)

In [ ]:
emp_ct <- c('Epiblast', 'Primitive Streak',  'Nascent mesoderm',  'Haematoendothelial progenitors',  'Embryo proper endothelium', 'Allantois endothelium',
            'Venous endothelium', 'Endocardium','YS endothelium',  'EMP', 'Megakaryocyte progenitors','MEP' )

In [ ]:
head(dfa)

In [ ]:
dfa_emp <- dfa[,colnames(dfa) %in% emp_ct]
dim(dfa_emp)

In [ ]:
head(dfa_emp)

In [ ]:
dfa_emp <- dfa[, emp_ct]

In [ ]:
head(dfa_emp)

In [ ]:
dfa_emp_s <- dfa_emp[KOs_and_paralogs,]
head(dfa_emp_s)

In [ ]:
dfa_emp_st <- t(dfa_emp_s)

In [ ]:
dfa_emp_st <- as.data.frame(dfa_emp_st)

In [ ]:
head(dfa_emp_st)

In [ ]:
dfa_emp_st$cluster <- rownames(dfa_emp_st)

In [ ]:
head(dfa_emp_st)

In [ ]:
str(dfa_emp_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Improved version for scientific publication
p_dfa_emp_st_atf3 <- ggplot(dfa_emp_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Atf3 "), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_emp_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Improved version for scientific publication
p_dfa_emp_st_zfp711 <- ggplot(dfa_emp_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_emp_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Improved version for scientific publication
p_dfa_emp_st_bcl6b <- ggplot(dfa_emp_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Bcl6b "), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_emp_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### in vitro subcluster_x (simplifed) (stay in the old version, not gonna use)

In [ ]:
table(wt_obj$new_subclusters_x)

In [ ]:
emp_ctx <- c('Epiblast', 'Primitive Streak',  'Nascent Mesoderm',  'HEP/BP',  'Endothelium', 'EMP')

In [ ]:
head(dfcx)

In [ ]:
dfcx_emp <- dfcx[,colnames(dfcx) %in% emp_ctx]
dim(dfcx_emp)

In [ ]:
head(dfcx_emp)

In [ ]:
dfcx_emp <- dfcx_emp[, emp_ctx]

In [ ]:
head(dfcx_emp)

In [ ]:
dfcx_emp_s <- dfcx_emp[KOs_and_paralogs,]
head(dfcx_emp_s)

In [ ]:
dfcx_emp_st <- t(dfcx_emp_s)

In [ ]:
dfcx_emp_st <- as.data.frame(dfcx_emp_st)

In [ ]:
head(dfcx_emp_st)

In [ ]:
dfcx_emp_st$cluster <- rownames(dfcx_emp_st)

In [ ]:
head(dfcx_emp_st)

In [ ]:
str(dfcx_emp_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 8, repr.plot.height = 6)
# Reorder clusters according to emp_ct vector
dfcx_emp_st$cluster <- factor(dfcx_emp_st$cluster, levels = emp_ctx)


# Alternative version with different styling
p_dfcx_emp_st_atf3 <- ggplot(dfcx_emp_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Atf3 Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

# Display alternative version
print(p_dfcx_emp_st_atf3)

In [ ]:
# Alternative version with different styling
p_dfcx_emp_st_zfp711 <- ggplot(dfcx_emp_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Zfp711 Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

# Display alternative version
print(p_dfcx_emp_st_zfp711)

In [ ]:
# Alternative version with different styling
p_dfcx_emp_st_bcl6b <- ggplot(dfcx_emp_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Bcl6b Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

# Display alternative version
print(p_dfcx_emp_st_bcl6b)

### in vitro subclusters (Detailed)

In [ ]:
table(wt_obj$new_subclusters)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
DimPlot(obj, reduction = "UMAP", group.by = 'new_subclusters', cols= r3dcol$cols_94)

In [ ]:
unique(obj$new_subclusters)

In [ ]:
emp_cts <- c('Epiblast', 'Primitive Streak 1',  'Primitive Streak 2', 'Nascent Mesoderm',  'HEP/BP',  'Endothelium 1', 'Endocardium','Endothelium 2','EMP 1', 'EMP 2', 'EMP 3', 'EMP 4','MkP 1','MkP 2','MEP')

In [ ]:
head(dfc)

In [ ]:
dfc_emp <- dfc[,colnames(dfc) %in% emp_cts]
dim(dfc_emp)

In [ ]:
head(dfc_emp)

In [ ]:
dfc_emp <- dfc_emp[, emp_cts]

In [ ]:
head(dfc_emp)

In [ ]:
dfc_emp_s <- dfc_emp[KOs_and_paralogs,]
head(dfc_emp_s)

In [ ]:
dfc_emp_st <- t(dfc_emp_s)

In [ ]:
dfc_emp_st <- as.data.frame(dfc_emp_st)

In [ ]:
head(dfc_emp_st)

In [ ]:
dfc_emp_st$cluster <- rownames(dfc_emp_st)

In [ ]:
head(dfc_emp_st)

In [ ]:
str(dfc_emp_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Improved version for scientific publication
p_dfc_emp_st_atf3 <- ggplot(dfc_emp_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Atf3 "), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_emp_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfc_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Improved version for scientific publication
p_dfc_emp_st_zfp711 <- ggplot(dfc_emp_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
    title = expression(paste(italic("In vitro"), " average ", italic("Zfp711"), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_emp_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfc_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Improved version for scientific publication
p_dfc_emp_st_bcl6b <- ggplot(dfc_emp_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
    title = expression(paste(italic("In vitro"), " average ", italic("Bcl6b "), "expression: second wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_emp_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfc_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 11)
p_second_wave <- plot_grid(
    p_dfa_emp_st_atf3,p_dfa_emp_st_zfp711,p_dfa_emp_st_bcl6b,
    p_dfc_emp_st_atf3,p_dfc_emp_st_zfp711,p_dfc_emp_st_bcl6b,
    
ncol = 3)
p_second_wave

### Paralogs

#### in vivo

In [ ]:
str(dfa_emp_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_emp_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_emp_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfa_emp_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_emp_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_emp_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfa_emp_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfa_emp_st$cluster <- factor(dfa_emp_st$cluster, levels = emp_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_emp_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_emp_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfa_emp_st_Bcl6b_paralogs)


#### in vitro simplified

In [ ]:
str(dfcx_emp_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfcx_emp_st$cluster <- factor(dfcx_emp_st$cluster, levels = emp_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_emp_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_emp_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_emp_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfcx_emp_st$cluster <- factor(dfcx_emp_st$cluster, levels = emp_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_emp_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_emp_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_emp_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfcx_emp_st$cluster <- factor(dfcx_emp_st$cluster, levels = emp_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_emp_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_emp_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_emp_st_Bcl6b_paralogs)


#### in vitro detailed

In [ ]:
str(dfc_emp_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_emp_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_emp_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_emp_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_emp_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_emp_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_emp_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to emp_ct vector
dfc_emp_st$cluster <- factor(dfc_emp_st$cluster, levels = emp_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_emp_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_emp_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_emp_st_Bcl6b_paralogs)


#### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 14)
plot_grid(
    p_dfa_emp_st_atf3_paralogs,p_dfa_emp_st_Zfp711_paralogs,p_dfa_emp_st_Bcl6b_paralogs,
    p_dfcx_emp_st_atf3_paralogs,p_dfcx_emp_st_Zfp711_paralogs,p_dfcx_emp_st_Bcl6b_paralogs,
    p_dfc_emp_st_atf3_paralogs,p_dfc_emp_st_Zfp711_paralogs,p_dfc_emp_st_Bcl6b_paralogs,
    
ncol = 3)

## erythroid

### in vivo

In [ ]:
erythroid_ct <- c('Epiblast', 'Primitive Streak',  'Nascent mesoderm',  'Haematoendothelial progenitors',  'Blood progenitors', 'Erythroid')

In [ ]:
head(dfa)

In [ ]:
dfa_erythroid <- dfa[,colnames(dfa) %in% erythroid_ct]
dim(dfa_erythroid)

In [ ]:
head(dfa_erythroid)

In [ ]:
dfa_erythroid <- dfa[, erythroid_ct]

In [ ]:
head(dfa_erythroid)

In [ ]:
dfa_erythroid_s <- dfa_erythroid[KOs_and_paralogs,]
head(dfa_erythroid_s)

In [ ]:
dfa_erythroid_st <- t(dfa_erythroid_s)

In [ ]:
dfa_erythroid_st <- as.data.frame(dfa_erythroid_st)

In [ ]:
head(dfa_erythroid_st)

In [ ]:
dfa_erythroid_st$cluster <- rownames(dfa_erythroid_st)

In [ ]:
head(dfa_erythroid_st)

In [ ]:
str(dfa_erythroid_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Improved version for scientific publication
p_dfa_erythroid_st_atf3 <- ggplot(dfa_erythroid_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Atf3 "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_erythroid_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Improved version for scientific publication
p_dfa_erythroid_st_zfp711 <- ggplot(dfa_erythroid_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_erythroid_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Improved version for scientific publication
p_dfa_erythroid_st_bcl6b <- ggplot(dfa_erythroid_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Bcl6b "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_erythroid_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### in vitro subcluster_x (simplifed)

In [ ]:
table(wt_obj$new_subclusters_x)

In [ ]:
erythroid_ctx <- c('Epiblast', 'Primitive Streak',  'Nascent Mesoderm',  'HEP/BP', 'Erythroid')

In [ ]:
head(dfcx)

In [ ]:
dfcx_erythroid <- dfcx[,colnames(dfcx) %in% erythroid_ctx]
dim(dfcx_erythroid)

In [ ]:
head(dfcx_erythroid)

In [ ]:
dfcx_erythroid <- dfcx_erythroid[, erythroid_ctx]

In [ ]:
head(dfcx_erythroid)

In [ ]:
dfcx_erythroid_s <- dfcx_erythroid[KOs_and_paralogs,]
head(dfcx_erythroid_s)

In [ ]:
dfcx_erythroid_st <- t(dfcx_erythroid_s)

In [ ]:
dfcx_erythroid_st <- as.data.frame(dfcx_erythroid_st)

In [ ]:
head(dfcx_erythroid_st)

In [ ]:
dfcx_erythroid_st$cluster <- rownames(dfcx_erythroid_st)

In [ ]:
head(dfcx_erythroid_st)

In [ ]:
str(dfcx_erythroid_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 8, repr.plot.height = 6)
# Reorder clusters according to erythroid_ct vector
dfcx_erythroid_st$cluster <- factor(dfcx_erythroid_st$cluster, levels = erythroid_ctx)


# Alternative version with different styling
p_dfcx_erythroid_st_atf3 <- ggplot(dfcx_erythroid_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Atf3 Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

# Display alternative version
print(p_dfcx_erythroid_st_atf3)

In [ ]:
# Alternative version with different styling
p_dfcx_erythroid_st_zfp711 <- ggplot(dfcx_erythroid_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Zfp711 Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

# Display alternative version
print(p_dfcx_erythroid_st_zfp711)

In [ ]:
# Alternative version with different styling
p_dfcx_erythroid_st_bcl6b <- ggplot(dfcx_erythroid_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", size = 1.5) +
  geom_point(color = "#ff7f0e", size = 4, stroke = 1.5, fill = "white", shape = 21) +
  labs(
    title = "Bcl6b Expression Profile",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
  )

options(repr.plot.width = 8, repr.plot.height = 6)
# Display alternative version
print(p_dfcx_erythroid_st_bcl6b)

### in vitro subclusters (Detailed)

In [ ]:
table(wt_obj$new_subclusters)

In [ ]:
erythroid_cts <- c('Epiblast', 'Primitive Streak 1',  'Primitive Streak 2', 'Nascent Mesoderm',  'HEP/BP', 'Erythroid 1', 'Erythroid 2', 'Erythroid 3', 'Erythroid 4')

In [ ]:
head(dfc)

In [ ]:
dfc_erythroid <- dfc[,colnames(dfc) %in% erythroid_cts]
dim(dfc_erythroid)

In [ ]:
head(dfc_erythroid)

In [ ]:
dfc_erythroid <- dfc_erythroid[, erythroid_cts]

In [ ]:
head(dfc_erythroid)

In [ ]:
dfc_erythroid_s <- dfc_erythroid[KOs_and_paralogs,]
head(dfc_erythroid_s)

In [ ]:
dfc_erythroid_st <- t(dfc_erythroid_s)

In [ ]:
dfc_erythroid_st <- as.data.frame(dfc_erythroid_st)

In [ ]:
head(dfc_erythroid_st)

In [ ]:
dfc_erythroid_st$cluster <- rownames(dfc_erythroid_st)

In [ ]:
head(dfc_erythroid_st)

In [ ]:
str(dfc_erythroid_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Improved version for scientific publication
p_dfc_erythroid_st_atf3 <- ggplot(dfc_erythroid_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Atf3 "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_erythroid_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Improved version for scientific publication
p_dfc_erythroid_st_zfp711 <- ggplot(dfc_erythroid_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Zfp711 "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_erythroid_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Improved version for scientific publication
p_dfc_erythroid_st_bcl6b <- ggplot(dfc_erythroid_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Bcl6b "), "expression: first wave")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_erythroid_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 11)
p_first_wave <- 
plot_grid(
    p_dfa_erythroid_st_atf3,p_dfa_erythroid_st_zfp711,p_dfa_erythroid_st_bcl6b,
    p_dfc_erythroid_st_atf3,p_dfc_erythroid_st_zfp711,p_dfc_erythroid_st_bcl6b,
    
ncol = 3)
p_first_wave

### Paralogs

#### in vivo

In [ ]:
str(dfa_erythroid_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_erythroid_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_erythroid_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))
options(repr.plot.width = 8, repr.plot.height = 6)
# Display the plot
print(p_dfa_erythroid_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_erythroid_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_erythroid_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfa_erythroid_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfa_erythroid_st$cluster <- factor(dfa_erythroid_st$cluster, levels = erythroid_ct)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfa_erythroid_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfa_erythroid_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfa_erythroid_st_Bcl6b_paralogs)


#### in vitro simplified

In [ ]:
str(dfcx_erythroid_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfcx_erythroid_st$cluster <- factor(dfcx_erythroid_st$cluster, levels = erythroid_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_erythroid_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_erythroid_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_erythroid_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfcx_erythroid_st$cluster <- factor(dfcx_erythroid_st$cluster, levels = erythroid_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_erythroid_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_erythroid_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_erythroid_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfcx_erythroid_st$cluster <- factor(dfcx_erythroid_st$cluster, levels = erythroid_ctx)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfcx_erythroid_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfcx_erythroid_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfcx_erythroid_st_Bcl6b_paralogs)


#### in vitro detailed

In [ ]:
str(dfc_erythroid_st)

In [ ]:
Atf3_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_erythroid_st %>%
  pivot_longer(cols = all_of(Atf3_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_erythroid_st_atf3_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "ATF3 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                                 "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                                 "Fosl2" = "#e377c2")) +
  scale_fill_manual(values = c("Atf3" = "#1f77b4", "Jdp2" = "#ff7f0e", "Batf3" = "#2ca02c", 
                               "Batf" = "#d62728", "Fosl1" = "#9467bd", "Fosb" = "#8c564b", 
                               "Fosl2" = "#e377c2")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_erythroid_st_atf3_paralogs)


In [ ]:
Zfp711_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_erythroid_st %>%
  pivot_longer(cols = all_of(Zfp711_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_erythroid_st_Zfp711_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Zfp711 Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                                 "Zfx" = "#d62728")) +
  scale_fill_manual(values = c("Zfp711" = "#1f77b4", "Zfy1" = "#ff7f0e", "Zfy2" = "#2ca02c", 
                               "Zfx" = "#d62728")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_erythroid_st_Zfp711_paralogs)


In [ ]:
Bcl6b_paralogs

In [ ]:
library(ggplot2)
library(dplyr)
library(tidyr)

# Reorder clusters according to erythroid_ct vector
dfc_erythroid_st$cluster <- factor(dfc_erythroid_st$cluster, levels = erythroid_cts)

# Reshape data to long format for ATF3 paralogs
dfa_long <- dfc_erythroid_st %>%
  pivot_longer(cols = all_of(Bcl6b_paralogs), 
               names_to = "gene", 
               values_to = "expression")

# Create multi-gene line plot
p_dfc_erythroid_st_Bcl6b_paralogs <- ggplot(dfa_long, aes(x = cluster, y = expression, group = gene, color = gene)) +
  geom_line(size = 1.5, alpha = 0.8) +
  geom_point(size = 4, stroke = 1.2, fill = "white", shape = 21, 
             aes(fill = gene), alpha = 0.9) +
  labs(
    title = "Bcl6b Paralog Expression Profiles",
    subtitle = "Across developmental cell clusters",
    x = "Developmental Stage",
    y = "Expression Level (normalized counts)",
    color = "Gene",
    fill = "Gene"
  ) +
  scale_color_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  scale_fill_manual(values = c("Bcl6b" = "#1f77b4", "Bcl6" = "#ff7f0e")) +
  theme_classic(base_size = 12) +
  theme(
    plot.title = element_text(size = 18, face = "bold", hjust = 0.5, color = "#2c3e50"),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray50"),
    axis.text.x = element_text(angle = 30, hjust = 1, size = 11),
    axis.title = element_text(size = 13, face = "bold"),
    axis.line = element_line(color = "#34495e", size = 0.8),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    legend.position = "right",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 11),
    legend.background = element_rect(fill = "white", color = "gray80"),
    legend.key = element_rect(fill = "white")
  ) +
  guides(color = guide_legend(override.aes = list(size = 3)),
         fill = guide_legend(override.aes = list(size = 3)))

# Display the plot
print(p_dfc_erythroid_st_Bcl6b_paralogs)


### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 14)
plot_grid(
    p_dfa_erythroid_st_atf3_paralogs,p_dfa_erythroid_st_Zfp711_paralogs,p_dfa_erythroid_st_Bcl6b_paralogs,
    p_dfcx_erythroid_st_atf3_paralogs,p_dfcx_erythroid_st_Zfp711_paralogs,p_dfcx_erythroid_st_Bcl6b_paralogs,
    p_dfc_erythroid_st_atf3_paralogs,p_dfc_erythroid_st_Zfp711_paralogs,p_dfc_erythroid_st_Bcl6b_paralogs,
    
ncol = 3)

## Late Mesoderm

### in vivo

In [ ]:
unique(atlas$celltype_extended_atlas)

In [ ]:
latemesoderm_ct <- c('Epiblast', 'Primitive Streak',  'Nascent mesoderm',  'Lateral plate mesoderm', 'Limb mesoderm',
                     'Mesenchyme','Allantois','Epicardium','YS mesothelium'
                    )

In [ ]:
head(dfa)

In [ ]:
dfa_latemesoderm <- dfa[,colnames(dfa) %in% latemesoderm_ct]
dim(dfa_latemesoderm)

In [ ]:
head(dfa_latemesoderm)

In [ ]:
dfa_latemesoderm <- dfa[, latemesoderm_ct]

In [ ]:
head(dfa_latemesoderm)

In [ ]:
dfa_latemesoderm_s <- dfa_latemesoderm[KOs_and_paralogs,]
head(dfa_latemesoderm_s)

In [ ]:
dfa_latemesoderm_st <- t(dfa_latemesoderm_s)

In [ ]:
dfa_latemesoderm_st <- as.data.frame(dfa_latemesoderm_st)

In [ ]:
head(dfa_latemesoderm_st)

In [ ]:
dfa_latemesoderm_st$cluster <- rownames(dfa_latemesoderm_st)

In [ ]:
head(dfa_latemesoderm_st)

In [ ]:
str(dfa_latemesoderm_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_latemesoderm_st$cluster <- factor(dfa_latemesoderm_st$cluster, levels = latemesoderm_ct)

# Improved version for scientific publication
p_dfa_latemesoderm_st_atf3 <- ggplot(dfa_latemesoderm_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Atf3 "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_latemesoderm_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_latemesoderm_st$cluster <- factor(dfa_latemesoderm_st$cluster, levels = latemesoderm_ct)

# Improved version for scientific publication
p_dfa_latemesoderm_st_zfp711 <- ggplot(dfa_latemesoderm_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_latemesoderm_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_latemesoderm_st$cluster <- factor(dfa_latemesoderm_st$cluster, levels = latemesoderm_ct)

# Improved version for scientific publication
p_dfa_latemesoderm_st_bcl6b <- ggplot(dfa_latemesoderm_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_latemesoderm_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### in vitro subclusters (Detailed)

In [ ]:
table(wt_obj$new_subclusters)

In [ ]:
latemesoderm_cts <- c('Epiblast', 'Primitive Streak 1',  'Primitive Streak 2', 'Nascent Mesoderm', 'Lateral Plate Mesoderm','Limb Mesoderm 1', 'Limb Mesoderm 2', 
                      'Mesenchyme 1','Mesenchyme 2',
                      'Allantois 1','Allantois 2','Allantois 3','Allantois 4',
                      'Epicardium 1','Epicardium 2', 'Epicardium 3')

In [ ]:
head(dfc)

In [ ]:
dfc_latemesoderm <- dfc[,colnames(dfc) %in% latemesoderm_cts]
dim(dfc_latemesoderm)

In [ ]:
head(dfc_latemesoderm)

In [ ]:
dfc_latemesoderm <- dfc_latemesoderm[, latemesoderm_cts]

In [ ]:
head(dfc_latemesoderm)

In [ ]:
dfc_latemesoderm_s <- dfc_latemesoderm[KOs_and_paralogs,]
head(dfc_latemesoderm_s)

In [ ]:
dfc_latemesoderm_st <- t(dfc_latemesoderm_s)

In [ ]:
dfc_latemesoderm_st <- as.data.frame(dfc_latemesoderm_st)

In [ ]:
head(dfc_latemesoderm_st)

In [ ]:
dfc_latemesoderm_st$cluster <- rownames(dfc_latemesoderm_st)

In [ ]:
head(dfc_latemesoderm_st)

In [ ]:
str(dfc_latemesoderm_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_latemesoderm_st$cluster <- factor(dfc_latemesoderm_st$cluster, levels = latemesoderm_cts)

# Improved version for scientific publication
p_dfc_latemesoderm_st_atf3 <- ggplot(dfc_latemesoderm_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Atf3 "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_latemesoderm_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_latemesoderm_st$cluster <- factor(dfc_latemesoderm_st$cluster, levels = latemesoderm_cts)

# Improved version for scientific publication
p_dfc_latemesoderm_st_zfp711 <- ggplot(dfc_latemesoderm_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Zfp711 "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_latemesoderm_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfc_latemesoderm_st$cluster <- factor(dfc_latemesoderm_st$cluster, levels = latemesoderm_cts)

# Improved version for scientific publication
p_dfc_latemesoderm_st_bcl6b <- ggplot(dfc_latemesoderm_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vitro"), " average ", italic("Bcl6b "), "expression: late mesoderm")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfc_latemesoderm_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 11)
p_late_mesoderm<- 
plot_grid(
    p_dfa_latemesoderm_st_atf3,p_dfa_latemesoderm_st_zfp711,p_dfa_latemesoderm_st_bcl6b,
    p_dfc_latemesoderm_st_atf3,p_dfc_latemesoderm_st_zfp711,p_dfc_latemesoderm_st_bcl6b,
    
ncol = 3)
p_late_mesoderm

## cardiac

### in vivo

In [ ]:
unique(atlas$celltype_extended_atlas)

In [ ]:
cardiac_ct <- c('Epiblast','Primitive Streak','Nascent mesoderm','Lateral plate mesoderm','Cardiopharyngeal progenitors',
                'Pharyngeal mesoderm','Anterior cardiopharyngeal progenitors','Cardiopharyngeal progenitors FHF', 
                'Cardiomyocytes FHF 1','Cardiomyocytes FHF 2', 
                'Cardiopharyngeal progenitors SHF','Cardiomyocytes SHF 1','Cardiomyocytes SHF 2')

In [ ]:
head(dfa)

In [ ]:
dfa_cardiac <- dfa[,colnames(dfa) %in% cardiac_ct]
dim(dfa_cardiac)

In [ ]:
head(dfa_cardiac)

In [ ]:
dfa_cardiac <- dfa[, cardiac_ct]

In [ ]:
head(dfa_cardiac)

In [ ]:
dfa_cardiac_s <- dfa_cardiac[KOs_and_paralogs,]
head(dfa_cardiac_s)

In [ ]:
dfa_cardiac_st <- t(dfa_cardiac_s)

In [ ]:
dfa_cardiac_st <- as.data.frame(dfa_cardiac_st)

In [ ]:
head(dfa_cardiac_st)

In [ ]:
dfa_cardiac_st$cluster <- rownames(dfa_cardiac_st)

In [ ]:
head(dfa_cardiac_st)

In [ ]:
str(dfa_cardiac_st)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_cardiac_st$cluster <- factor(dfa_cardiac_st$cluster, levels = cardiac_ct)

# Improved version for scientific publication
p_dfa_cardiac_st_atf3 <- ggplot(dfa_cardiac_st, aes(x = cluster, y = Atf3, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Atf3 "), "expression: cardiac lineages")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_cardiac_st_atf3)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_cardiac_st$cluster <- factor(dfa_cardiac_st$cluster, levels = cardiac_ct)

# Improved version for scientific publication
p_dfa_cardiac_st_zfp711 <- ggplot(dfa_cardiac_st, aes(x = cluster, y = Zfp711, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: cardiac lineages")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_cardiac_st_zfp711)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

In [ ]:
library(ggplot2)
library(dplyr)
options(repr.plot.width = 10, repr.plot.height = 6)

# Reorder clusters according to emp_ct vector
dfa_cardiac_st$cluster <- factor(dfa_cardiac_st$cluster, levels = cardiac_ct)

# Improved version for scientific publication
p_dfa_cardiac_st_bcl6b <- ggplot(dfa_cardiac_st, aes(x = cluster, y = Bcl6b, group = 1)) +
  geom_area(fill = "#E8F4FD", alpha = 0.3) +
  geom_line(color = "#1f77b4", linewidth = 1.2) +  # Updated to linewidth
  geom_point(color = "#ff7f0e", size = 3.5, stroke = 1.2, fill = "white", shape = 21) +
  labs(
   title = expression(paste(italic("In vivo"), " average ", italic("Zfp711 "), "expression: cardiac lineages")),
    x = "Ordered Developmental Stage - Cell Type",
    y = "Average Expression Level (normalized counts)"
  ) +
  theme_classic(base_size = 14) +  # Increased base font size
  theme(
    # Title improvements
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5, 
                             color = "black", margin = margin(b = 20)),
    
    # Axis improvements
    axis.title.x = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(t = 15)),
    axis.title.y = element_text(size = 12, face = "bold", color = "black",
                               margin = margin(r = 15)),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # Clean axis lines
    axis.line = element_line(color = "black", linewidth = 0.6),
    axis.ticks = element_line(color = "black", linewidth = 0.5),
    axis.ticks.length = unit(0.2, "cm"),
    
    # Background
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    
    # Margins for better spacing
    plot.margin = margin(20, 20, 20, 20),
    
    # Remove minor gridlines, keep major y-gridlines subtle
    panel.grid.major.y = element_line(color = "grey90", linewidth = 0.3),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  ) +
  # Add subtle y-axis breaks
  scale_y_continuous(expand = expansion(mult = c(0.02, 0.05)),
                     breaks = scales::pretty_breaks(n = 6))

# Display the plot
print(p_dfa_cardiac_st_bcl6b)

# Simple version without error bars or sample size annotations
# This is the clean version for publication without statistical elements
#p_final <- p_dfa_emp_st_atf3 +
#  labs(caption = NULL)  # Remove caption if not needed

#print(p_final)

### combined

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 11)
p_cardiac<- 
plot_grid(
    p_dfa_cardiac_st_atf3,p_dfa_cardiac_st_zfp711,p_dfa_cardiac_st_bcl6b,NULL, NULL, NULL,
    
ncol = 3)
p_cardiac

###### I am ready line plots

In [ ]:

p_line_1<- 
plot_grid(

    p_first_wave,
p_second_wave,
ncol = 1)


p_line_2<- 
plot_grid(
p_late_mesoderm,
p_cardiac,
ncol = 1)




In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22)
p_line_1

In [ ]:
pdf(file = './figures_3/plot_line_part1.pdf', width = 20, height = 22)
p_line_1
dev.off()

In [ ]:
svg(file = './figures_3/plot_line_part1.svg', width = 20, height = 22)
p_line_1
dev.off()

In [ ]:
png(file = './figures_3/plot_line_part1.png', width = 20, height = 22, units = 'in',res = 300)
p_line_1
dev.off()

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22)
p_line_2

In [ ]:
pdf(file = './figures_3/plot_line_part2.pdf', width = 20, height = 22)
p_line_2
dev.off()

In [ ]:
svg(file = './figures_3/plot_line_part2.svg', width = 20, height = 22)
p_line_2
dev.off()

In [ ]:
png(file = './figures_3/plot_line_part2.png', width = 20, height = 22, units = 'in',res = 300)
p_line_2
dev.off()

In [ ]:
ls()

In [ ]:
rm(atlas)

In [ ]:
rm(obj)

In [ ]:
gc()

In [ ]:
# 1. Find all object names containing "xobj"
objs_to_remove <- ls(envir = .GlobalEnv, pattern = "xobj")
objs_to_remove


In [ ]:
# 2. Remove them
if (length(objs_to_remove) > 0) {
  rm(list = objs_to_remove, envir = .GlobalEnv)
  message("Removed objects: ", paste(objs_to_remove, collapse = ", "))
} else {
  message("No objects matching 'xobj' found.")
}


In [ ]:
gc()

In [ ]:
ls()

In [ ]:
objs_to_remove <- ls(envir = .GlobalEnv, pattern = "curves")
objs_to_remove

# 2. Remove them
if (length(objs_to_remove) > 0) {
  rm(list = objs_to_remove, envir = .GlobalEnv)
  message("Removed objects: ", paste(objs_to_remove, collapse = ", "))
} else {
  message("No objects matching 'xobj' found.")
}


In [ ]:
objs_to_remove <- ls(envir = .GlobalEnv, pattern = "lineages")
objs_to_remove

# 2. Remove them
if (length(objs_to_remove) > 0) {
  rm(list = objs_to_remove, envir = .GlobalEnv)
  message("Removed objects: ", paste(objs_to_remove, collapse = ", "))
} else {
  message("No objects matching 'xobj' found.")
}


In [ ]:
gc()

In [ ]:
ls()

In [ ]:
save.image('./figure32.RData')

In [ ]:
sessionInfo()